# Ordering by metrics and retraining phase

## Dataset: Intel

## Experiment configuration 1


## Configuration 2
	2. Incremental guided retraining starting from the original model using the new adversarial inputs and original training set.

In [ ]:
pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
pwd

'C:\\Users\\fjdurlop\\Documents\\upc\\upc-july\\notebooks'

In [4]:
cd C:/Users/fjdurlop/Documents/upc/utils/

C:\Users\fjdurlop\Documents\upc\utils


In [5]:
# utils for project
import utils_guided_retraining2 as utils

2.5.0
keras
2.5.0


In [6]:
#cd '../notebooks/'

In [7]:
dataset = "cifar"

In [8]:
x_train,y_train = utils.get_data(dataset,"Train",verbose=True)
x_val,y_val = utils.get_data(dataset,"Val",verbose=True)
x_test,y_test = utils.get_data(dataset,"Test",verbose=True)

C:/Users/fjdurlop/Documents/upc/train_data/cifar/x_train.npy
x_set len:  41999
C:/Users/fjdurlop/Documents/upc/train_data/cifar/y_train.npy
y_set len:  41999
C:/Users/fjdurlop/Documents/upc/train_data/cifar/x_val.npy
x_set len:  12000
C:/Users/fjdurlop/Documents/upc/train_data/cifar/y_val.npy
y_set len:  12000
C:/Users/fjdurlop/Documents/upc/train_data/cifar/x_test.npy
x_set len:  6001
C:/Users/fjdurlop/Documents/upc/train_data/cifar/y_test.npy
y_set len:  6001


In [9]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary",verbose=True)

C:/Users/fjdurlop/Documents/upc/data_adversarial_july/cifar/train_and_adversary.npy
x_set len:  47999
C:/Users/fjdurlop/Documents/upc/data_adversarial_july/cifar/train_and_adversary_labels.npy
y_set len:  47999


In [10]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

6000
6000


In [11]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial') 

In [12]:
x_adversary_test = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test))
y_adversary_test = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test))

6000
6000


## ----

In [13]:
# Original model 

model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_02"


In [13]:
model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02"

In [14]:
print(model_dir)
model_original = utils.My_model(dataset,True, model_dir)

C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
Model loaded correctly


In [15]:
print(len(x_adversary_training))

6000


In [16]:
print(x_adversary_training.shape)
len(x_adversary_training)//20

(6000, 32, 32, 1)


300

In [17]:
n_data_points = 300

In [18]:
import gc

## Loading LSA and DSA values 

In [19]:
lsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"


In [20]:
lsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/lsa_values.npy"


In [21]:
lsa_values = np.load(lsa_direction)[len(x_train):] 
print(len(lsa_values))


6000


In [22]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_adversary_training)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_adversary_training)


In [23]:
m = n_data_points
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  300
300
1 :
0  ->  600
600
2 :
0  ->  900
900
3 :
0  ->  1200
1200
4 :
0  ->  1500
1500
5 :
0  ->  1800
1800
6 :
0  ->  2100
2100
7 :
0  ->  2400
2400
8 :
0  ->  2700
2700
9 :
0  ->  3000
3000
10 :
0  ->  3300
3300
11 :
0  ->  3600
3600
12 :
0  ->  3900
3900
13 :
0  ->  4200
4200
14 :
0  ->  4500
4500
15 :
0  ->  4800
4800
16 :
0  ->  5100
5100
17 :
0  ->  5400
5400
18 :
0  ->  5700
5700
19 :
Last
0  ->  6000
6000


In [24]:
len(image_sets_lsa)

20

## Training guided by LSA

In [25]:
len(label_sets_lsa)

20

In [26]:
print(model_dir)

C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02


In [27]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_lsa.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [28]:
models_lsa[0].evaluate(x_test,y_test)

188/188 [==============================] - 10s 10ms/step - loss: 0.5404 - accuracy: 0.8161


[0.524002730846405, 0.8258623480796814]

In [29]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n=n+1

Epoch 1/20
5/5 [==============================] - 7s 820ms/step - loss: 4.4029 - accuracy: 0.0867 - val_loss: 1.5072 - val_accuracy: 0.6246
Epoch 2/20
5/5 [==============================] - 3s 845ms/step - loss: 2.0991 - accuracy: 0.1800 - val_loss: 1.5096 - val_accuracy: 0.5640
Epoch 3/20
5/5 [==============================] - 3s 845ms/step - loss: 1.9463 - accuracy: 0.2000 - val_loss: 1.4928 - val_accuracy: 0.5170
Epoch 4/20
5/5 [==============================] - 4s 862ms/step - loss: 1.8753 - accuracy: 0.2567 - val_loss: 1.4630 - val_accuracy: 0.4888
Epoch 5/20
5/5 [==============================] - 4s 863ms/step - loss: 1.7310 - accuracy: 0.3333 - val_loss: 1.3833 - val_accuracy: 0.4341
Epoch 6/20
5/5 [==============================] - 4s 863ms/step - loss: 1.6759 - accuracy: 0.3400 - val_loss: 1.3363 - val_accuracy: 0.4699
Epoch 7/20
5/5 [==============================] - 4s 872ms/step - loss: 1.5261 - accuracy: 0.3800 - val_loss: 1.3662 - val_accuracy: 0.4504
Epoch 8/20
5/5 [====

In [30]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

Epoch 1/20
10/10 [==============================] - 10s 832ms/step - loss: 3.9629 - accuracy: 0.1623 - val_loss: 1.7010 - val_accuracy: 0.5197
Epoch 2/20
10/10 [==============================] - 7s 767ms/step - loss: 2.0960 - accuracy: 0.1825 - val_loss: 1.7411 - val_accuracy: 0.3616
Epoch 3/20
10/10 [==============================] - 7s 746ms/step - loss: 2.1249 - accuracy: 0.2167 - val_loss: 1.5496 - val_accuracy: 0.4974
Epoch 4/20
10/10 [==============================] - 7s 786ms/step - loss: 1.8628 - accuracy: 0.2676 - val_loss: 1.4867 - val_accuracy: 0.4890
Epoch 5/20
10/10 [==============================] - 7s 723ms/step - loss: 1.7989 - accuracy: 0.2958 - val_loss: 1.4563 - val_accuracy: 0.4851
Epoch 6/20
10/10 [==============================] - 7s 774ms/step - loss: 1.6450 - accuracy: 0.3704 - val_loss: 1.3346 - val_accuracy: 0.5385
Epoch 7/20
10/10 [==============================] - 7s 776ms/step - loss: 1.5905 - accuracy: 0.4070 - val_loss: 1.5503 - val_accuracy: 0.4223
Epoch

In [31]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
15/15 [==============================] - 12s 673ms/step - loss: 3.4963 - accuracy: 0.1416 - val_loss: 1.6406 - val_accuracy: 0.4601
Epoch 2/20
15/15 [==============================] - 8s 540ms/step - loss: 2.1451 - accuracy: 0.1940 - val_loss: 1.4752 - val_accuracy: 0.4931
Epoch 3/20
15/15 [==============================] - 7s 495ms/step - loss: 1.9509 - accuracy: 0.2961 - val_loss: 1.7162 - val_accuracy: 0.3447
Epoch 4/20
15/15 [==============================] - 7s 479ms/step - loss: 1.8945 - accuracy: 0.3349 - val_loss: 2.2504 - val_accuracy: 0.2061
Epoch 5/20
15/15 [==============================] - 7s 494ms/step - loss: 1.8711 - accuracy: 0.3128 - val_loss: 1.5491 - val_accuracy: 0.4184
Epoch 6/20
15/15 [==============================] - 7s 524ms/step - loss: 1.6486 - accuracy: 0.3830 - val_loss: 1.6947 - val_accuracy: 0.3643
Epoch 7/20
15/15 [==============================] - 7s 489ms/step - loss: 1.5901 - accuracy: 0.4038 - val_loss: 1.6969 - val_accuracy: 0.3673
Epo

In [32]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
19/19 [==============================] - 8s 389ms/step - loss: 3.5347 - accuracy: 0.1438 - val_loss: 1.6679 - val_accuracy: 0.4876
Epoch 2/20
19/19 [==============================] - 7s 375ms/step - loss: 1.9930 - accuracy: 0.2511 - val_loss: 1.6899 - val_accuracy: 0.3883
Epoch 3/20
19/19 [==============================] - 7s 379ms/step - loss: 1.8542 - accuracy: 0.2913 - val_loss: 1.4120 - val_accuracy: 0.5092
Epoch 4/20
19/19 [==============================] - 7s 370ms/step - loss: 1.6742 - accuracy: 0.3691 - val_loss: 1.5179 - val_accuracy: 0.4587
Epoch 5/20
19/19 [==============================] - 7s 380ms/step - loss: 1.6232 - accuracy: 0.4072 - val_loss: 1.3404 - val_accuracy: 0.5284
Epoch 6/20
19/19 [==============================] - 7s 372ms/step - loss: 1.5489 - accuracy: 0.4202 - val_loss: 1.3465 - val_accuracy: 0.5048
Epoch 7/20
19/19 [==============================] - 7s 372ms/step - loss: 1.4268 - accuracy: 0.4743 - val_loss: 1.3418 - val_accuracy: 0.5083
Epoc

In [33]:
print(n) #4
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
24/24 [==============================] - 9s 324ms/step - loss: 3.3724 - accuracy: 0.1340 - val_loss: 1.6457 - val_accuracy: 0.4910
Epoch 2/20
24/24 [==============================] - 7s 318ms/step - loss: 1.9864 - accuracy: 0.2601 - val_loss: 1.4738 - val_accuracy: 0.4855
Epoch 3/20
24/24 [==============================] - 7s 314ms/step - loss: 1.8882 - accuracy: 0.3080 - val_loss: 1.5071 - val_accuracy: 0.4810
Epoch 4/20
24/24 [==============================] - 7s 313ms/step - loss: 1.7226 - accuracy: 0.3423 - val_loss: 1.3544 - val_accuracy: 0.5352
Epoch 5/20
24/24 [==============================] - 7s 314ms/step - loss: 1.5665 - accuracy: 0.4166 - val_loss: 1.3900 - val_accuracy: 0.5028
Epoch 6/20
24/24 [==============================] - 7s 315ms/step - loss: 1.5010 - accuracy: 0.4289 - val_loss: 1.3643 - val_accuracy: 0.5037
Epoch 7/20
24/24 [==============================] - 7s 312ms/step - loss: 1.4670 - accuracy: 0.4599 - val_loss: 1.4087 - val_accuracy: 0.5341
Epoc

In [34]:
n=5
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
29/29 [==============================] - 9s 283ms/step - loss: 3.3236 - accuracy: 0.1604 - val_loss: 1.6707 - val_accuracy: 0.3941
Epoch 2/20
29/29 [==============================] - 8s 275ms/step - loss: 1.9452 - accuracy: 0.2848 - val_loss: 1.6661 - val_accuracy: 0.3869
Epoch 3/20
29/29 [==============================] - 8s 276ms/step - loss: 1.7743 - accuracy: 0.3602 - val_loss: 1.6181 - val_accuracy: 0.3882
Epoch 4/20
29/29 [==============================] - 8s 275ms/step - loss: 1.6387 - accuracy: 0.4118 - val_loss: 1.7060 - val_accuracy: 0.4182
Epoch 5/20
29/29 [==============================] - 8s 272ms/step - loss: 1.5770 - accuracy: 0.4212 - val_loss: 1.6504 - val_accuracy: 0.4053
Epoch 6/20
29/29 [==============================] - 8s 276ms/step - loss: 1.5028 - accuracy: 0.4622 - val_loss: 1.3684 - val_accuracy: 0.5042
Epoch 7/20
29/29 [==============================] - 8s 273ms/step - loss: 1.3398 - accuracy: 0.4630 - val_loss: 1.5256 - val_accuracy: 0.4353
Epoc

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
33/33 [==============================] - 10s 263ms/step - loss: 3.0493 - accuracy: 0.1545 - val_loss: 1.5526 - val_accuracy: 0.4650
Epoch 2/20
33/33 [==============================] - 8s 254ms/step - loss: 1.8897 - accuracy: 0.3005 - val_loss: 1.6323 - val_accuracy: 0.4030
Epoch 3/20
33/33 [==============================] - 8s 255ms/step - loss: 1.7263 - accuracy: 0.3715 - val_loss: 1.4218 - val_accuracy: 0.4822
Epoch 4/20
33/33 [==============================] - 8s 255ms/step - loss: 1.5403 - accuracy: 0.4143 - val_loss: 1.4107 - val_accuracy: 0.5151
Epoch 5/20
33/33 [==============================] - 8s 261ms/step - loss: 1.4698 - accuracy: 0.4467 - val_loss: 1.3223 - val_accuracy: 0.5466
Epoch 6/20
33/33 [==============================] - 8s 255ms/step - loss: 1.3707 - accuracy: 0.4923 - val_loss: 1.4056 - val_accuracy: 0.5074
Epoch 7/20
33/33 [==============================] - 8s 255ms/step - loss: 1.2380 - accuracy: 0.5366 - val_loss: 1.3428 - val_accuracy: 0.5351
Epo

In [36]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
38/38 [==============================] - 10s 241ms/step - loss: 2.9113 - accuracy: 0.2242 - val_loss: 1.3686 - val_accuracy: 0.5412
Epoch 2/20
38/38 [==============================] - 9s 235ms/step - loss: 1.7502 - accuracy: 0.3481 - val_loss: 1.4955 - val_accuracy: 0.5006
Epoch 3/20
38/38 [==============================] - 9s 235ms/step - loss: 1.6101 - accuracy: 0.4156 - val_loss: 1.3948 - val_accuracy: 0.5178
Epoch 4/20
38/38 [==============================] - 9s 234ms/step - loss: 1.4687 - accuracy: 0.4492 - val_loss: 1.3387 - val_accuracy: 0.5271
Epoch 5/20
38/38 [==============================] - 9s 234ms/step - loss: 1.3952 - accuracy: 0.4837 - val_loss: 1.2181 - val_accuracy: 0.5667
Epoch 6/20
38/38 [==============================] - 9s 235ms/step - loss: 1.2890 - accuracy: 0.5216 - val_loss: 1.3485 - val_accuracy: 0.5408
Epoch 7/20
38/38 [==============================] - 9s 253ms/step - loss: 1.1690 - accuracy: 0.5618 - val_loss: 1.1799 - val_accuracy: 0.5695
Epo

In [37]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
43/43 [==============================] - 11s 225ms/step - loss: 2.5737 - accuracy: 0.2514 - val_loss: 1.3270 - val_accuracy: 0.5567
Epoch 2/20
43/43 [==============================] - 9s 218ms/step - loss: 1.7535 - accuracy: 0.3376 - val_loss: 1.3449 - val_accuracy: 0.5344
Epoch 3/20
43/43 [==============================] - 9s 219ms/step - loss: 1.5746 - accuracy: 0.4194 - val_loss: 1.1750 - val_accuracy: 0.6024
Epoch 4/20
43/43 [==============================] - 9s 218ms/step - loss: 1.4850 - accuracy: 0.4466 - val_loss: 1.4158 - val_accuracy: 0.4725
Epoch 5/20
43/43 [==============================] - 9s 218ms/step - loss: 1.4094 - accuracy: 0.4733 - val_loss: 1.3066 - val_accuracy: 0.5651
Epoch 6/20
43/43 [==============================] - 9s 220ms/step - loss: 1.2882 - accuracy: 0.4970 - val_loss: 1.2469 - val_accuracy: 0.5637
Epoch 7/20
43/43 [==============================] - 9s 217ms/step - loss: 1.2415 - accuracy: 0.5367 - val_loss: 1.0992 - val_accuracy: 0.6126
Epo

In [38]:
print(n)#9
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
47/47 [==============================] - 12s 229ms/step - loss: 2.3785 - accuracy: 0.2803 - val_loss: 1.1704 - val_accuracy: 0.6841
Epoch 2/20
47/47 [==============================] - 10s 222ms/step - loss: 1.6659 - accuracy: 0.3583 - val_loss: 1.1424 - val_accuracy: 0.6349
Epoch 3/20
47/47 [==============================] - 10s 221ms/step - loss: 1.5224 - accuracy: 0.4307 - val_loss: 1.0988 - val_accuracy: 0.6639
Epoch 4/20
47/47 [==============================] - 10s 222ms/step - loss: 1.4514 - accuracy: 0.4463 - val_loss: 1.0608 - val_accuracy: 0.6750
Epoch 5/20
47/47 [==============================] - 10s 220ms/step - loss: 1.3682 - accuracy: 0.4782 - val_loss: 1.0862 - val_accuracy: 0.6450
Epoch 6/20
47/47 [==============================] - 10s 221ms/step - loss: 1.3190 - accuracy: 0.5070 - val_loss: 1.1266 - val_accuracy: 0.6406
Epoch 7/20
47/47 [==============================] - 11s 228ms/step - loss: 1.2419 - accuracy: 0.5337 - val_loss: 0.9200 - val_accuracy: 0.70

In [39]:
## print(n)
n=10
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

Epoch 1/20
52/52 [==============================] - 13s 217ms/step - loss: 2.4296 - accuracy: 0.2677 - val_loss: 1.1289 - val_accuracy: 0.6792
Epoch 2/20
52/52 [==============================] - 11s 207ms/step - loss: 1.6822 - accuracy: 0.3735 - val_loss: 1.3484 - val_accuracy: 0.5712
Epoch 3/20
52/52 [==============================] - 11s 209ms/step - loss: 1.5411 - accuracy: 0.4248 - val_loss: 1.0208 - val_accuracy: 0.6725
Epoch 4/20
52/52 [==============================] - 11s 210ms/step - loss: 1.4663 - accuracy: 0.4687 - val_loss: 0.9210 - val_accuracy: 0.6955
Epoch 5/20
52/52 [==============================] - 11s 208ms/step - loss: 1.3740 - accuracy: 0.4793 - val_loss: 1.0952 - val_accuracy: 0.6541
Epoch 6/20
52/52 [==============================] - 11s 208ms/step - loss: 1.2582 - accuracy: 0.5246 - val_loss: 0.9342 - val_accuracy: 0.6944
Epoch 7/20
52/52 [==============================] - 11s 208ms/step - loss: 1.2173 - accuracy: 0.5364 - val_loss: 1.0313 - val_accuracy: 0.6611

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
57/57 [==============================] - 13s 206ms/step - loss: 2.2063 - accuracy: 0.2940 - val_loss: 1.1963 - val_accuracy: 0.6283
Epoch 2/20
57/57 [==============================] - 11s 199ms/step - loss: 1.6280 - accuracy: 0.4008 - val_loss: 1.0620 - val_accuracy: 0.6478
Epoch 3/20
57/57 [==============================] - 11s 198ms/step - loss: 1.5870 - accuracy: 0.4256 - val_loss: 1.0480 - val_accuracy: 0.6708
Epoch 4/20
57/57 [==============================] - 11s 198ms/step - loss: 1.4586 - accuracy: 0.4594 - val_loss: 0.8679 - val_accuracy: 0.7243
Epoch 5/20
57/57 [==============================] - 11s 201ms/step - loss: 1.3639 - accuracy: 0.4953 - val_loss: 1.0913 - val_accuracy: 0.6469
Epoch 6/20
57/57 [==============================] - 11s 200ms/step - loss: 1.3148 - accuracy: 0.4965 - val_loss: 0.9572 - val_accuracy: 0.6802
Epoch 7/20
57/57 [==============================] - 11s 198ms/step - loss: 1.2368 - accuracy: 0.5426 - val_loss: 0.9498 - val_accuracy: 0.6

In [41]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
61/61 [==============================] - 14s 198ms/step - loss: 2.2487 - accuracy: 0.2972 - val_loss: 1.2124 - val_accuracy: 0.6818
Epoch 2/20
61/61 [==============================] - 12s 194ms/step - loss: 1.6160 - accuracy: 0.4064 - val_loss: 1.1724 - val_accuracy: 0.6246
Epoch 3/20
61/61 [==============================] - 12s 195ms/step - loss: 1.5189 - accuracy: 0.4234 - val_loss: 0.8872 - val_accuracy: 0.7212
Epoch 4/20
61/61 [==============================] - 12s 194ms/step - loss: 1.4163 - accuracy: 0.4726 - val_loss: 0.9572 - val_accuracy: 0.7130
Epoch 5/20
61/61 [==============================] - 12s 193ms/step - loss: 1.3269 - accuracy: 0.5031 - val_loss: 0.9650 - val_accuracy: 0.7025
Epoch 6/20
61/61 [==============================] - 12s 195ms/step - loss: 1.2876 - accuracy: 0.5117 - val_loss: 0.8975 - val_accuracy: 0.7079
Epoch 7/20
61/61 [==============================] - 12s 194ms/step - loss: 1.2054 - accuracy: 0.5601 - val_loss: 0.8609 - val_accuracy: 0.7

In [42]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
66/66 [==============================] - 14s 191ms/step - loss: 2.2701 - accuracy: 0.2841 - val_loss: 1.2168 - val_accuracy: 0.6390
Epoch 2/20
66/66 [==============================] - 12s 187ms/step - loss: 1.6303 - accuracy: 0.3953 - val_loss: 1.0851 - val_accuracy: 0.6713
Epoch 3/20
66/66 [==============================] - 12s 187ms/step - loss: 1.5300 - accuracy: 0.4398 - val_loss: 1.1686 - val_accuracy: 0.6535
Epoch 4/20
66/66 [==============================] - 12s 187ms/step - loss: 1.3463 - accuracy: 0.5013 - val_loss: 0.9735 - val_accuracy: 0.6850
Epoch 5/20
66/66 [==============================] - 12s 188ms/step - loss: 1.3127 - accuracy: 0.5199 - val_loss: 0.9568 - val_accuracy: 0.6896
Epoch 6/20
66/66 [==============================] - 12s 187ms/step - loss: 1.2299 - accuracy: 0.5551 - val_loss: 1.0904 - val_accuracy: 0.6418
Epoch 7/20
66/66 [==============================] - 12s 188ms/step - loss: 1.1734 - accuracy: 0.5849 - val_loss: 0.9893 - val_accuracy: 0.6

In [43]:
print(n)#14
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
71/71 [==============================] - 15s 185ms/step - loss: 2.2666 - accuracy: 0.2874 - val_loss: 1.2639 - val_accuracy: 0.5968
Epoch 2/20
71/71 [==============================] - 13s 181ms/step - loss: 1.6619 - accuracy: 0.3841 - val_loss: 0.9250 - val_accuracy: 0.7139
Epoch 3/20
71/71 [==============================] - 13s 181ms/step - loss: 1.5396 - accuracy: 0.4230 - val_loss: 0.8768 - val_accuracy: 0.7252
Epoch 4/20
71/71 [==============================] - 13s 180ms/step - loss: 1.4391 - accuracy: 0.4807 - val_loss: 1.1128 - val_accuracy: 0.6430
Epoch 5/20
71/71 [==============================] - 13s 181ms/step - loss: 1.3690 - accuracy: 0.4884 - val_loss: 0.9136 - val_accuracy: 0.7091
Epoch 6/20
71/71 [==============================] - 13s 181ms/step - loss: 1.2857 - accuracy: 0.5328 - val_loss: 1.0753 - val_accuracy: 0.6460
Epoch 7/20
71/71 [==============================] - 13s 182ms/step - loss: 1.1689 - accuracy: 0.5750 - val_loss: 0.9183 - val_accuracy: 0.6

In [44]:
n=15
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
75/75 [==============================] - 15s 181ms/step - loss: 2.1443 - accuracy: 0.3218 - val_loss: 1.0647 - val_accuracy: 0.6568
Epoch 2/20
75/75 [==============================] - 13s 176ms/step - loss: 1.5767 - accuracy: 0.4168 - val_loss: 0.9910 - val_accuracy: 0.6756
Epoch 3/20
75/75 [==============================] - 13s 177ms/step - loss: 1.4920 - accuracy: 0.4481 - val_loss: 0.8889 - val_accuracy: 0.7157
Epoch 4/20
75/75 [==============================] - 13s 175ms/step - loss: 1.3937 - accuracy: 0.4853 - val_loss: 0.9209 - val_accuracy: 0.7024
Epoch 5/20
75/75 [==============================] - 13s 177ms/step - loss: 1.3349 - accuracy: 0.5138 - val_loss: 0.8680 - val_accuracy: 0.7069
Epoch 6/20
75/75 [==============================] - 13s 177ms/step - loss: 1.2465 - accuracy: 0.5487 - val_loss: 0.8898 - val_accuracy: 0.7015
Epoch 7/20
75/75 [==============================] - 13s 178ms/step - loss: 1.1764 - accuracy: 0.5680 - val_loss: 1.0040 - val_accuracy: 0.6

In [45]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
80/80 [==============================] - 16s 177ms/step - loss: 2.2024 - accuracy: 0.3176 - val_loss: 1.0543 - val_accuracy: 0.6532
Epoch 2/20
80/80 [==============================] - 14s 173ms/step - loss: 1.5828 - accuracy: 0.4105 - val_loss: 1.0238 - val_accuracy: 0.6747
Epoch 3/20
80/80 [==============================] - 14s 173ms/step - loss: 1.4706 - accuracy: 0.4695 - val_loss: 0.9190 - val_accuracy: 0.6918
Epoch 4/20
80/80 [==============================] - 14s 172ms/step - loss: 1.4255 - accuracy: 0.4965 - val_loss: 1.0289 - val_accuracy: 0.6740
Epoch 5/20
80/80 [==============================] - 14s 173ms/step - loss: 1.3044 - accuracy: 0.5204 - val_loss: 0.8957 - val_accuracy: 0.7005
Epoch 6/20
80/80 [==============================] - 14s 173ms/step - loss: 1.2496 - accuracy: 0.5385 - val_loss: 0.9288 - val_accuracy: 0.6944
Epoch 7/20
80/80 [==============================] - 14s 173ms/step - loss: 1.1604 - accuracy: 0.5864 - val_loss: 0.8893 - val_accuracy: 0.6

In [46]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
85/85 [==============================] - 16s 173ms/step - loss: 2.1678 - accuracy: 0.3127 - val_loss: 0.9130 - val_accuracy: 0.6989
Epoch 2/20
85/85 [==============================] - 14s 169ms/step - loss: 1.6062 - accuracy: 0.4011 - val_loss: 0.9234 - val_accuracy: 0.7024
Epoch 3/20
85/85 [==============================] - 14s 169ms/step - loss: 1.5095 - accuracy: 0.4497 - val_loss: 1.2293 - val_accuracy: 0.5720
Epoch 4/20
85/85 [==============================] - 14s 169ms/step - loss: 1.3906 - accuracy: 0.4880 - val_loss: 0.8431 - val_accuracy: 0.7291
Epoch 5/20
85/85 [==============================] - 14s 168ms/step - loss: 1.3059 - accuracy: 0.5242 - val_loss: 0.8551 - val_accuracy: 0.7132
Epoch 6/20
85/85 [==============================] - 14s 169ms/step - loss: 1.2339 - accuracy: 0.5393 - val_loss: 0.9780 - val_accuracy: 0.6702
Epoch 7/20
85/85 [==============================] - 14s 169ms/step - loss: 1.1694 - accuracy: 0.5713 - val_loss: 0.8765 - val_accuracy: 0.7

In [48]:
time.sleep(400)

In [47]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
90/90 [==============================] - 17s 168ms/step - loss: 2.0711 - accuracy: 0.3280 - val_loss: 1.2318 - val_accuracy: 0.6003
Epoch 2/20
90/90 [==============================] - 15s 165ms/step - loss: 1.6110 - accuracy: 0.4078 - val_loss: 0.8668 - val_accuracy: 0.7313
Epoch 3/20
90/90 [==============================] - 15s 165ms/step - loss: 1.4946 - accuracy: 0.4636 - val_loss: 0.8503 - val_accuracy: 0.7148
Epoch 4/20
90/90 [==============================] - 15s 166ms/step - loss: 1.3849 - accuracy: 0.4857 - val_loss: 1.0060 - val_accuracy: 0.6393
Epoch 5/20
90/90 [==============================] - 15s 173ms/step - loss: 1.3335 - accuracy: 0.5246 - val_loss: 1.0111 - val_accuracy: 0.6532
Epoch 6/20
90/90 [==============================] - 16s 177ms/step - loss: 1.2641 - accuracy: 0.5417 - val_loss: 0.8223 - val_accuracy: 0.7277
Epoch 7/20
90/90 [==============================] - 16s 177ms/step - loss: 1.1951 - accuracy: 0.5590 - val_loss: 0.8960 - val_accuracy: 0.6

In [ ]:
time.sleep(400)

In [48]:
print(n)#19
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
94/94 [==============================] - 17s 166ms/step - loss: 1.9669 - accuracy: 0.3268 - val_loss: 0.8807 - val_accuracy: 0.7368
Epoch 2/20
94/94 [==============================] - 15s 163ms/step - loss: 1.5496 - accuracy: 0.4198 - val_loss: 0.8592 - val_accuracy: 0.7220
Epoch 3/20
94/94 [==============================] - 15s 163ms/step - loss: 1.4631 - accuracy: 0.4690 - val_loss: 0.8469 - val_accuracy: 0.7325
Epoch 4/20
94/94 [==============================] - 15s 163ms/step - loss: 1.3517 - accuracy: 0.5123 - val_loss: 0.8731 - val_accuracy: 0.7096
Epoch 5/20
94/94 [==============================] - 16s 166ms/step - loss: 1.2800 - accuracy: 0.5206 - val_loss: 0.7326 - val_accuracy: 0.7596
Epoch 6/20
94/94 [==============================] - 15s 162ms/step - loss: 1.2083 - accuracy: 0.5656 - val_loss: 0.8270 - val_accuracy: 0.7148
Epoch 7/20
94/94 [==============================] - 15s 163ms/step - loss: 1.1850 - accuracy: 0.5807 - val_loss: 0.8074 - val_accuracy: 0.7

In [49]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
new_model_lsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_lsa_e2"

i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_lsa_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_lsa_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_lsa_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_lsa_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_lsa_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_lsa_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_lsa_e2_6\assets
Model has been saved
INFO:t

In [50]:
import gc

In [51]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [52]:
gc.collect()

508555

## Training guided by DSA

In [53]:
dsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"

In [54]:
dsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/dsa_values.npy"

In [55]:
dsa_values = np.load(dsa_direction)[len(x_train):] 
print(len(dsa_values))

6000


In [56]:
top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_adversary_training)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_adversary_training)

In [57]:
m = n_data_points
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  300
300
1 :
0  ->  600
600
2 :
0  ->  900
900
3 :
0  ->  1200
1200
4 :
0  ->  1500
1500
5 :
0  ->  1800
1800
6 :
0  ->  2100
2100
7 :
0  ->  2400
2400
8 :
0  ->  2700
2700
9 :
0  ->  3000
3000
10 :
0  ->  3300
3300
11 :
0  ->  3600
3600
12 :
0  ->  3900
3900
13 :
0  ->  4200
4200
14 :
0  ->  4500
4500
15 :
0  ->  4800
4800
16 :
0  ->  5100
5100
17 :
0  ->  5400
5400
18 :
0  ->  5700
5700
19 :
Last
0  ->  6000
6000


In [58]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dsa.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [59]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
5/5 [==============================] - 9s 2s/step - loss: 5.4796 - accuracy: 0.0939 - val_loss: 1.3506 - val_accuracy: 0.5763
Epoch 2/20
5/5 [==============================] - 6s 2s/step - loss: 2.2193 - accuracy: 0.1510 - val_loss: 1.3640 - val_accuracy: 0.5845
Epoch 3/20
5/5 [==============================] - 6s 2s/step - loss: 2.0696 - accuracy: 0.1436 - val_loss: 1.3505 - val_accuracy: 0.5533
Epoch 4/20
5/5 [==============================] - 6s 2s/step - loss: 1.9597 - accuracy: 0.2217 - val_loss: 1.3898 - val_accuracy: 0.5293
Epoch 5/20
5/5 [==============================] - 6s 2s/step - loss: 1.8464 - accuracy: 0.3390 - val_loss: 1.3996 - val_accuracy: 0.5430
Epoch 6/20
5/5 [==============================] - 6s 2s/step - loss: 1.7301 - accuracy: 0.3184 - val_loss: 1.7555 - val_accuracy: 0.4588
Epoch 7/20
5/5 [==============================] - 6s 2s/step - loss: 1.6200 - accuracy: 0.3805 - val_loss: 1.8786 - val_accuracy: 0.5143
Epoch 8/20
5/5 [=======================

In [60]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
10/10 [==============================] - 8s 756ms/step - loss: 3.7811 - accuracy: 0.1664 - val_loss: 1.5637 - val_accuracy: 0.4952
Epoch 2/20
10/10 [==============================] - 7s 723ms/step - loss: 2.0785 - accuracy: 0.2483 - val_loss: 1.4657 - val_accuracy: 0.5603
Epoch 3/20
10/10 [==============================] - 7s 720ms/step - loss: 2.0253 - accuracy: 0.2354 - val_loss: 1.4727 - val_accuracy: 0.5329
Epoch 4/20
10/10 [==============================] - 7s 724ms/step - loss: 1.8504 - accuracy: 0.2973 - val_loss: 1.4116 - val_accuracy: 0.5025
Epoch 5/20
10/10 [==============================] - 7s 719ms/step - loss: 1.7238 - accuracy: 0.3325 - val_loss: 1.3977 - val_accuracy: 0.5229
Epoch 6/20
10/10 [==============================] - 7s 726ms/step - loss: 1.6326 - accuracy: 0.3763 - val_loss: 1.4537 - val_accuracy: 0.4852
Epoch 7/20
10/10 [==============================] - 7s 714ms/step - loss: 1.6044 - accuracy: 0.4165 - val_loss: 1.2664 - val_accuracy: 0.5553
Epoc

In [61]:
n=2
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
15/15 [==============================] - 9s 519ms/step - loss: 3.9211 - accuracy: 0.1199 - val_loss: 1.5626 - val_accuracy: 0.5882
Epoch 2/20
15/15 [==============================] - 7s 498ms/step - loss: 2.0949 - accuracy: 0.2142 - val_loss: 1.7075 - val_accuracy: 0.3820
Epoch 3/20
15/15 [==============================] - 7s 493ms/step - loss: 1.9313 - accuracy: 0.2731 - val_loss: 1.6574 - val_accuracy: 0.4133
Epoch 4/20
15/15 [==============================] - 7s 495ms/step - loss: 1.7910 - accuracy: 0.3349 - val_loss: 1.8060 - val_accuracy: 0.3643
Epoch 5/20
15/15 [==============================] - 7s 493ms/step - loss: 1.6893 - accuracy: 0.3738 - val_loss: 1.3351 - val_accuracy: 0.5667
Epoch 6/20
15/15 [==============================] - 7s 494ms/step - loss: 1.5415 - accuracy: 0.4261 - val_loss: 1.6137 - val_accuracy: 0.4569
Epoch 7/20
15/15 [==============================] - 7s 503ms/step - loss: 1.4333 - accuracy: 0.4739 - val_loss: 1.4523 - val_accuracy: 0.5109
Epoc

In [62]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
19/19 [==============================] - 10s 438ms/step - loss: 3.3074 - accuracy: 0.1717 - val_loss: 1.4173 - val_accuracy: 0.6273
Epoch 2/20
19/19 [==============================] - 8s 415ms/step - loss: 2.0069 - accuracy: 0.2387 - val_loss: 1.4934 - val_accuracy: 0.4706
Epoch 3/20
19/19 [==============================] - 8s 412ms/step - loss: 1.9213 - accuracy: 0.3040 - val_loss: 1.4888 - val_accuracy: 0.5085
Epoch 4/20
19/19 [==============================] - 8s 415ms/step - loss: 1.7241 - accuracy: 0.3438 - val_loss: 1.4580 - val_accuracy: 0.4931
Epoch 5/20
19/19 [==============================] - 8s 416ms/step - loss: 1.5781 - accuracy: 0.4316 - val_loss: 1.3564 - val_accuracy: 0.5321
Epoch 6/20
19/19 [==============================] - 8s 415ms/step - loss: 1.5193 - accuracy: 0.4233 - val_loss: 1.3821 - val_accuracy: 0.5168
Epoch 7/20
19/19 [==============================] - 8s 416ms/step - loss: 1.3084 - accuracy: 0.5163 - val_loss: 1.3986 - val_accuracy: 0.5220
Epo

In [63]:
print(n)#4

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
24/24 [==============================] - 10s 360ms/step - loss: 3.0499 - accuracy: 0.1519 - val_loss: 1.5768 - val_accuracy: 0.5468
Epoch 2/20
24/24 [==============================] - 8s 345ms/step - loss: 2.0005 - accuracy: 0.2536 - val_loss: 1.4510 - val_accuracy: 0.5140
Epoch 3/20
24/24 [==============================] - 8s 347ms/step - loss: 1.8781 - accuracy: 0.2940 - val_loss: 1.4982 - val_accuracy: 0.4916
Epoch 4/20
24/24 [==============================] - 8s 344ms/step - loss: 1.6737 - accuracy: 0.3741 - val_loss: 1.3253 - val_accuracy: 0.5518
Epoch 5/20
24/24 [==============================] - 8s 348ms/step - loss: 1.5988 - accuracy: 0.4296 - val_loss: 1.4077 - val_accuracy: 0.5448
Epoch 6/20
24/24 [==============================] - 8s 346ms/step - loss: 1.4849 - accuracy: 0.4528 - val_loss: 1.3380 - val_accuracy: 0.5557
Epoch 7/20
24/24 [==============================] - 8s 344ms/step - loss: 1.3337 - accuracy: 0.4912 - val_loss: 1.3176 - val_accuracy: 0.5623
Epo

In [64]:
n=5
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
29/29 [==============================] - 11s 313ms/step - loss: 2.8327 - accuracy: 0.1961 - val_loss: 1.5795 - val_accuracy: 0.4860
Epoch 2/20
29/29 [==============================] - 9s 302ms/step - loss: 1.8831 - accuracy: 0.3139 - val_loss: 1.5653 - val_accuracy: 0.4709
Epoch 3/20
29/29 [==============================] - 9s 301ms/step - loss: 1.7356 - accuracy: 0.3662 - val_loss: 1.3736 - val_accuracy: 0.5333
Epoch 4/20
29/29 [==============================] - 9s 304ms/step - loss: 1.6589 - accuracy: 0.3940 - val_loss: 1.5690 - val_accuracy: 0.4560
Epoch 5/20
29/29 [==============================] - 9s 303ms/step - loss: 1.4775 - accuracy: 0.4601 - val_loss: 1.3405 - val_accuracy: 0.5338
Epoch 6/20
29/29 [==============================] - 9s 300ms/step - loss: 1.3609 - accuracy: 0.4955 - val_loss: 1.3194 - val_accuracy: 0.5381
Epoch 7/20
29/29 [==============================] - 9s 301ms/step - loss: 1.2984 - accuracy: 0.5273 - val_loss: 1.3124 - val_accuracy: 0.5408
Epo

In [65]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
33/33 [==============================] - 11s 287ms/step - loss: 2.8809 - accuracy: 0.1669 - val_loss: 1.6389 - val_accuracy: 0.4351
Epoch 2/20
33/33 [==============================] - 9s 278ms/step - loss: 1.9170 - accuracy: 0.3106 - val_loss: 1.4260 - val_accuracy: 0.5486
Epoch 3/20
33/33 [==============================] - 9s 278ms/step - loss: 1.6564 - accuracy: 0.4026 - val_loss: 1.3575 - val_accuracy: 0.5415
Epoch 4/20
33/33 [==============================] - 9s 277ms/step - loss: 1.5641 - accuracy: 0.4406 - val_loss: 1.3742 - val_accuracy: 0.5249
Epoch 5/20
33/33 [==============================] - 9s 279ms/step - loss: 1.4438 - accuracy: 0.4822 - val_loss: 1.2573 - val_accuracy: 0.5714
Epoch 6/20
33/33 [==============================] - 9s 278ms/step - loss: 1.3145 - accuracy: 0.5349 - val_loss: 1.1880 - val_accuracy: 0.5956
Epoch 7/20
33/33 [==============================] - 9s 279ms/step - loss: 1.2243 - accuracy: 0.5526 - val_loss: 1.2862 - val_accuracy: 0.5525
Epo

In [66]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
38/38 [==============================] - 11s 263ms/step - loss: 2.7056 - accuracy: 0.1677 - val_loss: 1.3892 - val_accuracy: 0.5357
Epoch 2/20
38/38 [==============================] - 10s 256ms/step - loss: 1.8737 - accuracy: 0.3217 - val_loss: 1.4089 - val_accuracy: 0.5501
Epoch 3/20
38/38 [==============================] - 10s 256ms/step - loss: 1.6502 - accuracy: 0.4101 - val_loss: 1.3523 - val_accuracy: 0.5450
Epoch 4/20
38/38 [==============================] - 10s 260ms/step - loss: 1.5148 - accuracy: 0.4511 - val_loss: 1.3657 - val_accuracy: 0.5598
Epoch 5/20
38/38 [==============================] - 10s 260ms/step - loss: 1.3864 - accuracy: 0.5024 - val_loss: 1.2765 - val_accuracy: 0.5492
Epoch 6/20
38/38 [==============================] - 10s 254ms/step - loss: 1.2839 - accuracy: 0.5393 - val_loss: 1.2315 - val_accuracy: 0.5829
Epoch 7/20
38/38 [==============================] - 10s 255ms/step - loss: 1.1925 - accuracy: 0.5816 - val_loss: 1.2992 - val_accuracy: 0.55

In [67]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
43/43 [==============================] - 12s 245ms/step - loss: 2.8024 - accuracy: 0.1668 - val_loss: 1.5130 - val_accuracy: 0.4542
Epoch 2/20
43/43 [==============================] - 10s 239ms/step - loss: 1.8542 - accuracy: 0.3127 - val_loss: 1.4362 - val_accuracy: 0.4981
Epoch 3/20
43/43 [==============================] - 10s 239ms/step - loss: 1.6808 - accuracy: 0.3831 - val_loss: 1.3265 - val_accuracy: 0.5562
Epoch 4/20
43/43 [==============================] - 10s 240ms/step - loss: 1.4738 - accuracy: 0.4565 - val_loss: 1.3212 - val_accuracy: 0.5552
Epoch 5/20
43/43 [==============================] - 10s 242ms/step - loss: 1.3836 - accuracy: 0.5174 - val_loss: 1.3011 - val_accuracy: 0.5548
Epoch 6/20
43/43 [==============================] - 10s 237ms/step - loss: 1.3046 - accuracy: 0.5246 - val_loss: 1.2625 - val_accuracy: 0.5706
Epoch 7/20
43/43 [==============================] - 10s 237ms/step - loss: 1.1863 - accuracy: 0.5777 - val_loss: 1.3319 - val_accuracy: 0.53

In [68]:
print(n)#9

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
47/47 [==============================] - 13s 237ms/step - loss: 2.7126 - accuracy: 0.1564 - val_loss: 1.5671 - val_accuracy: 0.5029
Epoch 2/20
47/47 [==============================] - 11s 228ms/step - loss: 1.8850 - accuracy: 0.3146 - val_loss: 1.5042 - val_accuracy: 0.5164
Epoch 3/20
47/47 [==============================] - 11s 227ms/step - loss: 1.6659 - accuracy: 0.3953 - val_loss: 1.3081 - val_accuracy: 0.5781
Epoch 4/20
47/47 [==============================] - 11s 229ms/step - loss: 1.5132 - accuracy: 0.4612 - val_loss: 1.1951 - val_accuracy: 0.5918
Epoch 5/20
47/47 [==============================] - 11s 227ms/step - loss: 1.4033 - accuracy: 0.4973 - val_loss: 1.1727 - val_accuracy: 0.5961
Epoch 6/20
47/47 [==============================] - 11s 229ms/step - loss: 1.2860 - accuracy: 0.5507 - val_loss: 1.1896 - val_accuracy: 0.5951
Epoch 7/20
47/47 [==============================] - 11s 228ms/step - loss: 1.2349 - accuracy: 0.5590 - val_loss: 1.1196 - val_accuracy: 0.61

In [69]:
n=10
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
52/52 [==============================] - 13s 224ms/step - loss: 2.8158 - accuracy: 0.1521 - val_loss: 1.5789 - val_accuracy: 0.4659
Epoch 2/20
52/52 [==============================] - 11s 216ms/step - loss: 1.8634 - accuracy: 0.3150 - val_loss: 1.4129 - val_accuracy: 0.5368
Epoch 3/20
52/52 [==============================] - 11s 216ms/step - loss: 1.6365 - accuracy: 0.4114 - val_loss: 1.2987 - val_accuracy: 0.5758
Epoch 4/20
52/52 [==============================] - 11s 218ms/step - loss: 1.4563 - accuracy: 0.4826 - val_loss: 1.3285 - val_accuracy: 0.5607
Epoch 5/20
52/52 [==============================] - 11s 216ms/step - loss: 1.3681 - accuracy: 0.5282 - val_loss: 1.2608 - val_accuracy: 0.5624
Epoch 6/20
52/52 [==============================] - 11s 216ms/step - loss: 1.2357 - accuracy: 0.5568 - val_loss: 1.0814 - val_accuracy: 0.6263
Epoch 7/20
52/52 [==============================] - 11s 216ms/step - loss: 1.1742 - accuracy: 0.5920 - val_loss: 1.1727 - val_accuracy: 0.6

In [70]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
57/57 [==============================] - 14s 214ms/step - loss: 2.5845 - accuracy: 0.1768 - val_loss: 1.6411 - val_accuracy: 0.4369
Epoch 2/20
57/57 [==============================] - 12s 208ms/step - loss: 1.7915 - accuracy: 0.3450 - val_loss: 1.3978 - val_accuracy: 0.5417
Epoch 3/20
57/57 [==============================] - 12s 209ms/step - loss: 1.5902 - accuracy: 0.4222 - val_loss: 1.1693 - val_accuracy: 0.6068
Epoch 4/20
57/57 [==============================] - 12s 207ms/step - loss: 1.4629 - accuracy: 0.4748 - val_loss: 1.4014 - val_accuracy: 0.5302
Epoch 5/20
57/57 [==============================] - 12s 207ms/step - loss: 1.3779 - accuracy: 0.5041 - val_loss: 1.1071 - val_accuracy: 0.6222
Epoch 6/20
57/57 [==============================] - 12s 206ms/step - loss: 1.2696 - accuracy: 0.5446 - val_loss: 1.2455 - val_accuracy: 0.5682
Epoch 7/20
57/57 [==============================] - 12s 211ms/step - loss: 1.1503 - accuracy: 0.5806 - val_loss: 1.1686 - val_accuracy: 0.5

In [71]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
61/61 [==============================] - 14s 210ms/step - loss: 2.6074 - accuracy: 0.1591 - val_loss: 1.5564 - val_accuracy: 0.4898
Epoch 2/20
61/61 [==============================] - 12s 202ms/step - loss: 1.7602 - accuracy: 0.3584 - val_loss: 1.4756 - val_accuracy: 0.5228
Epoch 3/20
61/61 [==============================] - 12s 203ms/step - loss: 1.5792 - accuracy: 0.4279 - val_loss: 1.2143 - val_accuracy: 0.6093
Epoch 4/20
61/61 [==============================] - 12s 203ms/step - loss: 1.4457 - accuracy: 0.4863 - val_loss: 1.3003 - val_accuracy: 0.5727
Epoch 5/20
61/61 [==============================] - 12s 204ms/step - loss: 1.3040 - accuracy: 0.5375 - val_loss: 1.2131 - val_accuracy: 0.6042
Epoch 6/20
61/61 [==============================] - 13s 212ms/step - loss: 1.2567 - accuracy: 0.5431 - val_loss: 1.1681 - val_accuracy: 0.6231
Epoch 7/20
61/61 [==============================] - 13s 215ms/step - loss: 1.1494 - accuracy: 0.5837 - val_loss: 1.1297 - val_accuracy: 0.6

In [72]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
66/66 [==============================] - 15s 201ms/step - loss: 2.5098 - accuracy: 0.2033 - val_loss: 1.4342 - val_accuracy: 0.5001
Epoch 2/20
66/66 [==============================] - 13s 196ms/step - loss: 1.7662 - accuracy: 0.3691 - val_loss: 1.3445 - val_accuracy: 0.5602
Epoch 3/20
66/66 [==============================] - 13s 196ms/step - loss: 1.5569 - accuracy: 0.4377 - val_loss: 1.2874 - val_accuracy: 0.5891
Epoch 4/20
66/66 [==============================] - 13s 197ms/step - loss: 1.4527 - accuracy: 0.4724 - val_loss: 1.1446 - val_accuracy: 0.6123
Epoch 5/20
66/66 [==============================] - 13s 195ms/step - loss: 1.3561 - accuracy: 0.5278 - val_loss: 1.2103 - val_accuracy: 0.5817
Epoch 6/20
66/66 [==============================] - 13s 196ms/step - loss: 1.2258 - accuracy: 0.5620 - val_loss: 1.1679 - val_accuracy: 0.6034
Epoch 7/20
66/66 [==============================] - 13s 196ms/step - loss: 1.1477 - accuracy: 0.5825 - val_loss: 1.1191 - val_accuracy: 0.6

In [73]:
print(n)#14

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
71/71 [==============================] - 15s 194ms/step - loss: 2.4919 - accuracy: 0.2004 - val_loss: 1.4085 - val_accuracy: 0.5295
Epoch 2/20
71/71 [==============================] - 13s 191ms/step - loss: 1.7531 - accuracy: 0.3568 - val_loss: 1.1963 - val_accuracy: 0.5674
Epoch 3/20
71/71 [==============================] - 14s 196ms/step - loss: 1.5686 - accuracy: 0.4410 - val_loss: 1.2672 - val_accuracy: 0.5891
Epoch 4/20
71/71 [==============================] - 13s 190ms/step - loss: 1.4095 - accuracy: 0.4921 - val_loss: 1.1112 - val_accuracy: 0.6391
Epoch 5/20
71/71 [==============================] - 14s 201ms/step - loss: 1.3101 - accuracy: 0.5258 - val_loss: 1.1628 - val_accuracy: 0.6065
Epoch 6/20
71/71 [==============================] - 13s 181ms/step - loss: 1.2318 - accuracy: 0.5557 - val_loss: 1.0675 - val_accuracy: 0.6266
Epoch 7/20
71/71 [==============================] - 13s 184ms/step - loss: 1.1455 - accuracy: 0.5971 - val_loss: 1.0497 - val_accuracy: 0.6

In [74]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
75/75 [==============================] - 17s 211ms/step - loss: 2.3899 - accuracy: 0.2322 - val_loss: 1.2663 - val_accuracy: 0.5694
Epoch 2/20
75/75 [==============================] - 14s 188ms/step - loss: 1.6977 - accuracy: 0.3805 - val_loss: 1.0660 - val_accuracy: 0.6312
Epoch 3/20
75/75 [==============================] - 16s 208ms/step - loss: 1.5449 - accuracy: 0.4400 - val_loss: 1.0992 - val_accuracy: 0.6331
Epoch 4/20
75/75 [==============================] - 16s 214ms/step - loss: 1.4368 - accuracy: 0.4790 - val_loss: 1.1102 - val_accuracy: 0.6280
Epoch 5/20
75/75 [==============================] - 16s 218ms/step - loss: 1.3082 - accuracy: 0.5272 - val_loss: 0.9968 - val_accuracy: 0.6622
Epoch 6/20
75/75 [==============================] - 17s 223ms/step - loss: 1.2670 - accuracy: 0.5510 - val_loss: 1.0359 - val_accuracy: 0.6578
Epoch 7/20
75/75 [==============================] - 16s 214ms/step - loss: 1.1765 - accuracy: 0.5756 - val_loss: 0.9748 - val_accuracy: 0.6

In [75]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
80/80 [==============================] - 18s 210ms/step - loss: 2.2706 - accuracy: 0.2642 - val_loss: 1.0373 - val_accuracy: 0.6696
Epoch 2/20
80/80 [==============================] - 15s 186ms/step - loss: 1.7528 - accuracy: 0.3440 - val_loss: 0.8716 - val_accuracy: 0.6983
Epoch 3/20
80/80 [==============================] - 17s 215ms/step - loss: 1.5871 - accuracy: 0.4262 - val_loss: 0.9597 - val_accuracy: 0.6891
Epoch 4/20
80/80 [==============================] - 17s 211ms/step - loss: 1.4875 - accuracy: 0.4646 - val_loss: 0.9494 - val_accuracy: 0.6923
Epoch 5/20
80/80 [==============================] - 15s 194ms/step - loss: 1.4096 - accuracy: 0.4916 - val_loss: 0.9101 - val_accuracy: 0.6840
Epoch 6/20
80/80 [==============================] - 16s 195ms/step - loss: 1.3186 - accuracy: 0.5142 - val_loss: 0.9717 - val_accuracy: 0.6495
Epoch 7/20
80/80 [==============================] - 15s 190ms/step - loss: 1.2636 - accuracy: 0.5460 - val_loss: 0.8510 - val_accuracy: 0.7

In [77]:
time.sleep(400)

In [76]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
85/85 [==============================] - 17s 183ms/step - loss: 2.2088 - accuracy: 0.2716 - val_loss: 1.1051 - val_accuracy: 0.6567
Epoch 2/20
85/85 [==============================] - 16s 185ms/step - loss: 1.6522 - accuracy: 0.3852 - val_loss: 0.9448 - val_accuracy: 0.6877
Epoch 3/20
85/85 [==============================] - 15s 172ms/step - loss: 1.5611 - accuracy: 0.4146 - val_loss: 0.9211 - val_accuracy: 0.6827
Epoch 4/20
85/85 [==============================] - 15s 174ms/step - loss: 1.4400 - accuracy: 0.4687 - val_loss: 0.8573 - val_accuracy: 0.7118
Epoch 5/20
85/85 [==============================] - 15s 174ms/step - loss: 1.3742 - accuracy: 0.5005 - val_loss: 0.8319 - val_accuracy: 0.7153
Epoch 6/20
85/85 [==============================] - 16s 194ms/step - loss: 1.2734 - accuracy: 0.5385 - val_loss: 1.0206 - val_accuracy: 0.6622
Epoch 7/20
85/85 [==============================] - 15s 174ms/step - loss: 1.2509 - accuracy: 0.5491 - val_loss: 0.8043 - val_accuracy: 0.7

In [79]:
time.sleep(400)

In [77]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
90/90 [==============================] - 22s 218ms/step - loss: 2.2081 - accuracy: 0.2927 - val_loss: 0.8452 - val_accuracy: 0.7374
Epoch 2/20
90/90 [==============================] - 16s 173ms/step - loss: 1.6488 - accuracy: 0.3871 - val_loss: 1.0226 - val_accuracy: 0.6903
Epoch 3/20
90/90 [==============================] - 15s 171ms/step - loss: 1.5373 - accuracy: 0.4239 - val_loss: 0.9558 - val_accuracy: 0.6563
Epoch 4/20
90/90 [==============================] - 15s 167ms/step - loss: 1.4716 - accuracy: 0.4734 - val_loss: 0.8247 - val_accuracy: 0.7248
Epoch 5/20
90/90 [==============================] - 15s 166ms/step - loss: 1.3808 - accuracy: 0.4943 - val_loss: 0.8804 - val_accuracy: 0.7153
Epoch 6/20
90/90 [==============================] - 15s 165ms/step - loss: 1.2763 - accuracy: 0.5376 - val_loss: 0.8168 - val_accuracy: 0.7326
Epoch 7/20
90/90 [==============================] - 17s 185ms/step - loss: 1.2613 - accuracy: 0.5485 - val_loss: 1.1266 - val_accuracy: 0.6

In [ ]:
time.sleep(400)

In [78]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
94/94 [==============================] - 19s 183ms/step - loss: 2.0015 - accuracy: 0.3303 - val_loss: 0.8164 - val_accuracy: 0.7282
Epoch 2/20
94/94 [==============================] - 20s 212ms/step - loss: 1.5998 - accuracy: 0.4236 - val_loss: 0.7820 - val_accuracy: 0.7697
Epoch 3/20
94/94 [==============================] - 17s 176ms/step - loss: 1.4580 - accuracy: 0.4744 - val_loss: 0.7611 - val_accuracy: 0.7657
Epoch 4/20
94/94 [==============================] - 16s 173ms/step - loss: 1.3932 - accuracy: 0.5005 - val_loss: 0.8332 - val_accuracy: 0.7339
Epoch 5/20
94/94 [==============================] - 16s 171ms/step - loss: 1.3050 - accuracy: 0.5347 - val_loss: 0.7591 - val_accuracy: 0.7575
Epoch 6/20
94/94 [==============================] - 16s 173ms/step - loss: 1.2522 - accuracy: 0.5375 - val_loss: 0.8671 - val_accuracy: 0.7382
Epoch 7/20
94/94 [==============================] - 16s 170ms/step - loss: 1.1847 - accuracy: 0.5757 - val_loss: 0.8077 - val_accuracy: 0.7

In [ ]:
time.sleep(400)

In [79]:
new_model_dsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_dsa_e2"


i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dsa_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dsa_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dsa_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dsa_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dsa_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dsa_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dsa_e2_6\assets
Model has been saved
INFO:t

In [80]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [81]:
gc.collect()

508542

## Uncertainty

In [82]:
dg_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)[len(x_train):] 
print(len(deep_gini_values))


6000


In [83]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_adversary_training)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_adversary_training)

In [84]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  300
300
1 :
0  ->  600
600
2 :
0  ->  900
900
3 :
0  ->  1200
1200
4 :
0  ->  1500
1500
5 :
0  ->  1800
1800
6 :
0  ->  2100
2100
7 :
0  ->  2400
2400
8 :
0  ->  2700
2700
9 :
0  ->  3000
3000
10 :
0  ->  3300
3300
11 :
0  ->  3600
3600
12 :
0  ->  3900
3900
13 :
0  ->  4200
4200
14 :
0  ->  4500
4500
15 :
0  ->  4800
4800
16 :
0  ->  5100
5100
17 :
0  ->  5400
5400
18 :
0  ->  5700
5700
19 :
Last
0  ->  6000
6000


In [85]:
print(model_dir)
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dg.append(model)

C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [86]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
5/5 [==============================] - 8s 2s/step - loss: 2.4983 - accuracy: 0.2402 - val_loss: 1.1059 - val_accuracy: 0.6883
Epoch 2/20
5/5 [==============================] - 6s 1s/step - loss: 1.7710 - accuracy: 0.3392 - val_loss: 1.0565 - val_accuracy: 0.6663
Epoch 3/20
5/5 [==============================] - 6s 1s/step - loss: 1.6013 - accuracy: 0.4249 - val_loss: 0.9692 - val_accuracy: 0.7065
Epoch 4/20
5/5 [==============================] - 6s 1s/step - loss: 1.4203 - accuracy: 0.5013 - val_loss: 1.0810 - val_accuracy: 0.6845
Epoch 5/20
5/5 [==============================] - 6s 1s/step - loss: 1.4202 - accuracy: 0.4972 - val_loss: 0.8374 - val_accuracy: 0.7240
Epoch 6/20
5/5 [==============================] - 6s 1s/step - loss: 1.2663 - accuracy: 0.5491 - val_loss: 0.7751 - val_accuracy: 0.7554
Epoch 7/20
5/5 [==============================] - 6s 1s/step - loss: 1.0571 - accuracy: 0.6276 - val_loss: 0.8548 - val_accuracy: 0.7253
Epoch 8/20
5/5 [=======================

In [87]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
10/10 [==============================] - 8s 699ms/step - loss: 2.4020 - accuracy: 0.2059 - val_loss: 1.3910 - val_accuracy: 0.5573
Epoch 2/20
10/10 [==============================] - 6s 666ms/step - loss: 1.8769 - accuracy: 0.3195 - val_loss: 1.2021 - val_accuracy: 0.6797
Epoch 3/20
10/10 [==============================] - 6s 669ms/step - loss: 1.6993 - accuracy: 0.3969 - val_loss: 1.1584 - val_accuracy: 0.6749
Epoch 4/20
10/10 [==============================] - 6s 668ms/step - loss: 1.7182 - accuracy: 0.4159 - val_loss: 1.0904 - val_accuracy: 0.6706
Epoch 5/20
10/10 [==============================] - 6s 661ms/step - loss: 1.4386 - accuracy: 0.5017 - val_loss: 1.0072 - val_accuracy: 0.6765
Epoch 6/20
10/10 [==============================] - 6s 669ms/step - loss: 1.3665 - accuracy: 0.5405 - val_loss: 1.0631 - val_accuracy: 0.6428
Epoch 7/20
10/10 [==============================] - 6s 662ms/step - loss: 1.3323 - accuracy: 0.5181 - val_loss: 0.9022 - val_accuracy: 0.7164
Epoc

In [88]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
15/15 [==============================] - 9s 519ms/step - loss: 2.5557 - accuracy: 0.2196 - val_loss: 1.2068 - val_accuracy: 0.6658
Epoch 2/20
15/15 [==============================] - 7s 484ms/step - loss: 1.8182 - accuracy: 0.3594 - val_loss: 1.4551 - val_accuracy: 0.5918
Epoch 3/20
15/15 [==============================] - 7s 481ms/step - loss: 1.8366 - accuracy: 0.3178 - val_loss: 1.2261 - val_accuracy: 0.6313
Epoch 4/20
15/15 [==============================] - 8s 529ms/step - loss: 1.6352 - accuracy: 0.4124 - val_loss: 1.0336 - val_accuracy: 0.6698
Epoch 5/20
15/15 [==============================] - 8s 531ms/step - loss: 1.5137 - accuracy: 0.4717 - val_loss: 1.1038 - val_accuracy: 0.6224
Epoch 6/20
15/15 [==============================] - 7s 507ms/step - loss: 1.3326 - accuracy: 0.5208 - val_loss: 0.9620 - val_accuracy: 0.6848
Epoch 7/20
15/15 [==============================] - 7s 522ms/step - loss: 1.2591 - accuracy: 0.5587 - val_loss: 1.1793 - val_accuracy: 0.5942
Epoc

In [89]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
19/19 [==============================] - 10s 460ms/step - loss: 2.4784 - accuracy: 0.2280 - val_loss: 1.1898 - val_accuracy: 0.6821
Epoch 2/20
19/19 [==============================] - 8s 433ms/step - loss: 1.8231 - accuracy: 0.3368 - val_loss: 1.0965 - val_accuracy: 0.6626
Epoch 3/20
19/19 [==============================] - 7s 404ms/step - loss: 1.6068 - accuracy: 0.4297 - val_loss: 1.0973 - val_accuracy: 0.6712
Epoch 4/20
19/19 [==============================] - 8s 412ms/step - loss: 1.5350 - accuracy: 0.4650 - val_loss: 1.0609 - val_accuracy: 0.6472
Epoch 5/20
19/19 [==============================] - 8s 413ms/step - loss: 1.4630 - accuracy: 0.4840 - val_loss: 1.0405 - val_accuracy: 0.6482
Epoch 6/20
19/19 [==============================] - 8s 415ms/step - loss: 1.2865 - accuracy: 0.5179 - val_loss: 1.0067 - val_accuracy: 0.6683
Epoch 7/20
19/19 [==============================] - 8s 427ms/step - loss: 1.2165 - accuracy: 0.5729 - val_loss: 1.0120 - val_accuracy: 0.6698
Epo

In [90]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
24/24 [==============================] - 10s 347ms/step - loss: 2.6415 - accuracy: 0.1923 - val_loss: 1.1106 - val_accuracy: 0.6612
Epoch 2/20
24/24 [==============================] - 8s 339ms/step - loss: 1.8896 - accuracy: 0.3266 - val_loss: 1.0716 - val_accuracy: 0.6530
Epoch 3/20
24/24 [==============================] - 8s 355ms/step - loss: 1.6379 - accuracy: 0.3976 - val_loss: 1.0530 - val_accuracy: 0.6532
Epoch 4/20
24/24 [==============================] - 8s 353ms/step - loss: 1.5097 - accuracy: 0.4341 - val_loss: 1.0365 - val_accuracy: 0.6603
Epoch 5/20
24/24 [==============================] - 8s 352ms/step - loss: 1.4210 - accuracy: 0.5027 - val_loss: 1.1215 - val_accuracy: 0.6339
Epoch 6/20
24/24 [==============================] - 8s 354ms/step - loss: 1.4729 - accuracy: 0.4606 - val_loss: 1.0745 - val_accuracy: 0.6330
Epoch 7/20
24/24 [==============================] - 8s 352ms/step - loss: 1.3053 - accuracy: 0.5349 - val_loss: 1.2297 - val_accuracy: 0.5829
Epo

In [91]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
29/29 [==============================] - 10s 297ms/step - loss: 2.4607 - accuracy: 0.2090 - val_loss: 1.2941 - val_accuracy: 0.5889
Epoch 2/20
29/29 [==============================] - 8s 290ms/step - loss: 1.8619 - accuracy: 0.3248 - val_loss: 1.0779 - val_accuracy: 0.6821
Epoch 3/20
29/29 [==============================] - 8s 285ms/step - loss: 1.6913 - accuracy: 0.3813 - val_loss: 1.3781 - val_accuracy: 0.5344
Epoch 4/20
29/29 [==============================] - 8s 287ms/step - loss: 1.6045 - accuracy: 0.4156 - val_loss: 1.1632 - val_accuracy: 0.6183
Epoch 5/20
29/29 [==============================] - 8s 291ms/step - loss: 1.5210 - accuracy: 0.4702 - val_loss: 1.1208 - val_accuracy: 0.6242
Epoch 6/20
29/29 [==============================] - 8s 287ms/step - loss: 1.3696 - accuracy: 0.5174 - val_loss: 1.1581 - val_accuracy: 0.6371
Epoch 7/20
29/29 [==============================] - 8s 285ms/step - loss: 1.2833 - accuracy: 0.5544 - val_loss: 1.0967 - val_accuracy: 0.6279
Epo

In [92]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
33/33 [==============================] - 10s 276ms/step - loss: 2.5140 - accuracy: 0.2122 - val_loss: 1.5579 - val_accuracy: 0.5265
Epoch 2/20
33/33 [==============================] - 9s 266ms/step - loss: 1.8418 - accuracy: 0.3216 - val_loss: 1.1964 - val_accuracy: 0.6402
Epoch 3/20
33/33 [==============================] - 9s 266ms/step - loss: 1.6427 - accuracy: 0.4150 - val_loss: 1.2298 - val_accuracy: 0.5763
Epoch 4/20
33/33 [==============================] - 9s 271ms/step - loss: 1.5627 - accuracy: 0.4405 - val_loss: 1.1755 - val_accuracy: 0.6119
Epoch 5/20
33/33 [==============================] - 9s 265ms/step - loss: 1.4571 - accuracy: 0.4837 - val_loss: 1.1808 - val_accuracy: 0.6153
Epoch 6/20
33/33 [==============================] - 9s 268ms/step - loss: 1.2652 - accuracy: 0.5614 - val_loss: 1.1817 - val_accuracy: 0.6024
Epoch 7/20
33/33 [==============================] - 9s 264ms/step - loss: 1.2456 - accuracy: 0.5706 - val_loss: 1.0785 - val_accuracy: 0.6417
Epo

In [93]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
38/38 [==============================] - 11s 254ms/step - loss: 2.3643 - accuracy: 0.2237 - val_loss: 1.2057 - val_accuracy: 0.6568
Epoch 2/20
38/38 [==============================] - 9s 245ms/step - loss: 1.8273 - accuracy: 0.3296 - val_loss: 1.1719 - val_accuracy: 0.6526
Epoch 3/20
38/38 [==============================] - 9s 249ms/step - loss: 1.6076 - accuracy: 0.4129 - val_loss: 1.1601 - val_accuracy: 0.6144
Epoch 4/20
38/38 [==============================] - 9s 247ms/step - loss: 1.4865 - accuracy: 0.4738 - val_loss: 1.0664 - val_accuracy: 0.6557
Epoch 5/20
38/38 [==============================] - 9s 247ms/step - loss: 1.3954 - accuracy: 0.4926 - val_loss: 1.0894 - val_accuracy: 0.6505
Epoch 6/20
38/38 [==============================] - 9s 245ms/step - loss: 1.2615 - accuracy: 0.5452 - val_loss: 1.0934 - val_accuracy: 0.6531
Epoch 7/20
38/38 [==============================] - 9s 246ms/step - loss: 1.1504 - accuracy: 0.5872 - val_loss: 1.0213 - val_accuracy: 0.6577
Epo

In [94]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
43/43 [==============================] - 12s 249ms/step - loss: 2.4021 - accuracy: 0.2464 - val_loss: 1.1860 - val_accuracy: 0.5984
Epoch 2/20
43/43 [==============================] - 10s 232ms/step - loss: 1.7861 - accuracy: 0.3496 - val_loss: 1.1050 - val_accuracy: 0.6514
Epoch 3/20
43/43 [==============================] - 10s 235ms/step - loss: 1.6327 - accuracy: 0.4247 - val_loss: 1.0584 - val_accuracy: 0.6582
Epoch 4/20
43/43 [==============================] - 10s 230ms/step - loss: 1.5502 - accuracy: 0.4475 - val_loss: 1.0845 - val_accuracy: 0.6284
Epoch 5/20
43/43 [==============================] - 10s 228ms/step - loss: 1.3510 - accuracy: 0.5227 - val_loss: 1.0833 - val_accuracy: 0.6307
Epoch 6/20
43/43 [==============================] - 10s 228ms/step - loss: 1.3337 - accuracy: 0.5325 - val_loss: 1.0987 - val_accuracy: 0.6384
Epoch 7/20
43/43 [==============================] - 10s 226ms/step - loss: 1.2420 - accuracy: 0.5624 - val_loss: 1.0595 - val_accuracy: 0.64

In [95]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
47/47 [==============================] - 12s 223ms/step - loss: 2.5152 - accuracy: 0.2281 - val_loss: 1.0574 - val_accuracy: 0.6932
Epoch 2/20
47/47 [==============================] - 10s 221ms/step - loss: 1.8023 - accuracy: 0.3417 - val_loss: 1.1441 - val_accuracy: 0.6568
Epoch 3/20
47/47 [==============================] - 10s 220ms/step - loss: 1.6822 - accuracy: 0.3827 - val_loss: 1.0976 - val_accuracy: 0.6825
Epoch 4/20
47/47 [==============================] - 10s 219ms/step - loss: 1.5872 - accuracy: 0.4249 - val_loss: 0.9242 - val_accuracy: 0.7081
Epoch 5/20
47/47 [==============================] - 10s 218ms/step - loss: 1.4848 - accuracy: 0.4758 - val_loss: 0.9487 - val_accuracy: 0.7028
Epoch 6/20
47/47 [==============================] - 10s 220ms/step - loss: 1.3558 - accuracy: 0.5065 - val_loss: 0.8464 - val_accuracy: 0.7207
Epoch 7/20
47/47 [==============================] - 10s 220ms/step - loss: 1.2827 - accuracy: 0.5483 - val_loss: 0.9460 - val_accuracy: 0.69

In [96]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
52/52 [==============================] - 13s 215ms/step - loss: 2.4665 - accuracy: 0.2245 - val_loss: 1.3172 - val_accuracy: 0.6181
Epoch 2/20
52/52 [==============================] - 13s 253ms/step - loss: 1.7729 - accuracy: 0.3491 - val_loss: 1.2572 - val_accuracy: 0.6272
Epoch 3/20
52/52 [==============================] - 13s 257ms/step - loss: 1.6032 - accuracy: 0.4303 - val_loss: 1.1912 - val_accuracy: 0.6401
Epoch 4/20
52/52 [==============================] - 12s 231ms/step - loss: 1.5023 - accuracy: 0.4666 - val_loss: 1.0770 - val_accuracy: 0.6419
Epoch 5/20
52/52 [==============================] - 12s 226ms/step - loss: 1.3661 - accuracy: 0.5191 - val_loss: 1.0598 - val_accuracy: 0.6583
Epoch 6/20
52/52 [==============================] - 12s 230ms/step - loss: 1.3013 - accuracy: 0.5252 - val_loss: 0.9526 - val_accuracy: 0.6803
Epoch 7/20
52/52 [==============================] - 12s 227ms/step - loss: 1.1763 - accuracy: 0.5834 - val_loss: 1.0251 - val_accuracy: 0.6

In [97]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
57/57 [==============================] - 14s 216ms/step - loss: 2.3545 - accuracy: 0.2561 - val_loss: 1.1681 - val_accuracy: 0.6382
Epoch 2/20
57/57 [==============================] - 12s 208ms/step - loss: 1.7658 - accuracy: 0.3608 - val_loss: 1.1903 - val_accuracy: 0.5849
Epoch 3/20
57/57 [==============================] - 12s 215ms/step - loss: 1.6086 - accuracy: 0.4142 - val_loss: 1.2708 - val_accuracy: 0.6205
Epoch 4/20
57/57 [==============================] - 12s 209ms/step - loss: 1.4517 - accuracy: 0.4731 - val_loss: 1.0205 - val_accuracy: 0.6513
Epoch 5/20
57/57 [==============================] - 12s 211ms/step - loss: 1.3878 - accuracy: 0.5040 - val_loss: 1.1993 - val_accuracy: 0.6082
Epoch 6/20
57/57 [==============================] - 12s 211ms/step - loss: 1.2691 - accuracy: 0.5430 - val_loss: 1.0518 - val_accuracy: 0.6463
Epoch 7/20
57/57 [==============================] - 12s 208ms/step - loss: 1.1804 - accuracy: 0.5718 - val_loss: 1.0094 - val_accuracy: 0.6

In [98]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
61/61 [==============================] - 15s 220ms/step - loss: 2.4088 - accuracy: 0.2481 - val_loss: 1.1826 - val_accuracy: 0.6631
Epoch 2/20
61/61 [==============================] - 12s 207ms/step - loss: 1.7465 - accuracy: 0.3770 - val_loss: 1.1511 - val_accuracy: 0.6617
Epoch 3/20
61/61 [==============================] - 12s 202ms/step - loss: 1.5741 - accuracy: 0.4432 - val_loss: 1.0871 - val_accuracy: 0.6480
Epoch 4/20
61/61 [==============================] - 13s 209ms/step - loss: 1.4740 - accuracy: 0.4530 - val_loss: 0.9388 - val_accuracy: 0.6872
Epoch 5/20
61/61 [==============================] - 12s 206ms/step - loss: 1.3823 - accuracy: 0.5140 - val_loss: 0.9178 - val_accuracy: 0.6986
Epoch 6/20
61/61 [==============================] - 12s 206ms/step - loss: 1.2763 - accuracy: 0.5254 - val_loss: 0.9195 - val_accuracy: 0.7053
Epoch 7/20
61/61 [==============================] - 12s 206ms/step - loss: 1.2095 - accuracy: 0.5728 - val_loss: 0.9504 - val_accuracy: 0.6

In [99]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
66/66 [==============================] - 15s 203ms/step - loss: 2.2753 - accuracy: 0.2611 - val_loss: 1.0332 - val_accuracy: 0.7067
Epoch 2/20
66/66 [==============================] - 13s 198ms/step - loss: 1.7098 - accuracy: 0.3707 - val_loss: 1.1255 - val_accuracy: 0.6718
Epoch 3/20
66/66 [==============================] - 13s 198ms/step - loss: 1.5843 - accuracy: 0.4142 - val_loss: 0.9926 - val_accuracy: 0.6783
Epoch 4/20
66/66 [==============================] - 13s 198ms/step - loss: 1.4536 - accuracy: 0.4736 - val_loss: 1.0310 - val_accuracy: 0.6742
Epoch 5/20
66/66 [==============================] - 13s 205ms/step - loss: 1.3315 - accuracy: 0.5220 - val_loss: 1.0371 - val_accuracy: 0.6870
Epoch 6/20
66/66 [==============================] - 14s 209ms/step - loss: 1.2633 - accuracy: 0.5488 - val_loss: 0.8651 - val_accuracy: 0.7199
Epoch 7/20
66/66 [==============================] - 13s 199ms/step - loss: 1.1870 - accuracy: 0.5591 - val_loss: 0.9286 - val_accuracy: 0.6

In [100]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
71/71 [==============================] - 16s 204ms/step - loss: 2.2770 - accuracy: 0.2554 - val_loss: 1.0993 - val_accuracy: 0.6563
Epoch 2/20
71/71 [==============================] - 14s 197ms/step - loss: 1.6998 - accuracy: 0.3872 - val_loss: 1.2088 - val_accuracy: 0.6258
Epoch 3/20
71/71 [==============================] - 14s 194ms/step - loss: 1.5423 - accuracy: 0.4452 - val_loss: 1.0434 - val_accuracy: 0.6763
Epoch 4/20
71/71 [==============================] - 13s 191ms/step - loss: 1.4402 - accuracy: 0.4837 - val_loss: 1.0593 - val_accuracy: 0.6728
Epoch 5/20
71/71 [==============================] - 14s 192ms/step - loss: 1.3430 - accuracy: 0.5138 - val_loss: 1.1538 - val_accuracy: 0.6393
Epoch 6/20
71/71 [==============================] - 14s 201ms/step - loss: 1.2913 - accuracy: 0.5272 - val_loss: 0.9727 - val_accuracy: 0.6952
Epoch 7/20
71/71 [==============================] - 14s 199ms/step - loss: 1.1918 - accuracy: 0.5828 - val_loss: 1.0074 - val_accuracy: 0.6

In [101]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
75/75 [==============================] - 16s 196ms/step - loss: 2.3698 - accuracy: 0.2491 - val_loss: 1.1525 - val_accuracy: 0.6653
Epoch 2/20
75/75 [==============================] - 15s 199ms/step - loss: 1.6973 - accuracy: 0.3843 - val_loss: 1.0491 - val_accuracy: 0.6912
Epoch 3/20
75/75 [==============================] - 14s 193ms/step - loss: 1.5684 - accuracy: 0.4329 - val_loss: 1.0456 - val_accuracy: 0.6805
Epoch 4/20
75/75 [==============================] - 14s 188ms/step - loss: 1.4490 - accuracy: 0.4646 - val_loss: 1.0232 - val_accuracy: 0.6732
Epoch 5/20
75/75 [==============================] - 14s 190ms/step - loss: 1.3848 - accuracy: 0.5024 - val_loss: 1.0333 - val_accuracy: 0.6732
Epoch 6/20
75/75 [==============================] - 14s 191ms/step - loss: 1.2797 - accuracy: 0.5438 - val_loss: 0.9772 - val_accuracy: 0.6999
Epoch 7/20
75/75 [==============================] - 14s 191ms/step - loss: 1.2197 - accuracy: 0.5694 - val_loss: 0.9529 - val_accuracy: 0.6

In [102]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
80/80 [==============================] - 16s 186ms/step - loss: 2.2595 - accuracy: 0.2685 - val_loss: 1.1078 - val_accuracy: 0.7128
Epoch 2/20
80/80 [==============================] - 15s 183ms/step - loss: 1.7279 - accuracy: 0.3680 - val_loss: 0.9340 - val_accuracy: 0.7083
Epoch 3/20
80/80 [==============================] - 15s 185ms/step - loss: 1.5613 - accuracy: 0.4250 - val_loss: 0.9427 - val_accuracy: 0.7000
Epoch 4/20
80/80 [==============================] - 15s 184ms/step - loss: 1.5025 - accuracy: 0.4505 - val_loss: 0.7903 - val_accuracy: 0.7368
Epoch 5/20
80/80 [==============================] - 14s 181ms/step - loss: 1.4389 - accuracy: 0.4783 - val_loss: 0.9094 - val_accuracy: 0.7192
Epoch 6/20
80/80 [==============================] - 15s 182ms/step - loss: 1.3336 - accuracy: 0.5128 - val_loss: 0.9191 - val_accuracy: 0.6952
Epoch 7/20
80/80 [==============================] - 15s 184ms/step - loss: 1.2637 - accuracy: 0.5325 - val_loss: 0.8139 - val_accuracy: 0.7

In [103]:
time.sleep(400)

In [104]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
85/85 [==============================] - 11s 115ms/step - loss: 2.2508 - accuracy: 0.2768 - val_loss: 0.9357 - val_accuracy: 0.7284
Epoch 2/20
85/85 [==============================] - 10s 119ms/step - loss: 1.6964 - accuracy: 0.3775 - val_loss: 0.9831 - val_accuracy: 0.6883
Epoch 3/20
85/85 [==============================] - 10s 120ms/step - loss: 1.5447 - accuracy: 0.4317 - val_loss: 0.8516 - val_accuracy: 0.7242
Epoch 4/20
85/85 [==============================] - 12s 142ms/step - loss: 1.4555 - accuracy: 0.4669 - val_loss: 0.7985 - val_accuracy: 0.7487
Epoch 5/20
85/85 [==============================] - 14s 162ms/step - loss: 1.3740 - accuracy: 0.5023 - val_loss: 0.7769 - val_accuracy: 0.7430
Epoch 6/20
85/85 [==============================] - 16s 188ms/step - loss: 1.3026 - accuracy: 0.5298 - val_loss: 0.7759 - val_accuracy: 0.7548
Epoch 7/20
85/85 [==============================] - 17s 202ms/step - loss: 1.2628 - accuracy: 0.5471 - val_loss: 0.8285 - val_accuracy: 0.7

In [ ]:
time.sleep(400)

In [105]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
90/90 [==============================] - 18s 184ms/step - loss: 2.0534 - accuracy: 0.3168 - val_loss: 1.0527 - val_accuracy: 0.6787
Epoch 2/20
90/90 [==============================] - 17s 186ms/step - loss: 1.6178 - accuracy: 0.4085 - val_loss: 1.0436 - val_accuracy: 0.6324
Epoch 3/20
90/90 [==============================] - 16s 179ms/step - loss: 1.5177 - accuracy: 0.4485 - val_loss: 0.9669 - val_accuracy: 0.6841
Epoch 4/20
90/90 [==============================] - 17s 194ms/step - loss: 1.4398 - accuracy: 0.4710 - val_loss: 0.9727 - val_accuracy: 0.6593
Epoch 5/20
90/90 [==============================] - 16s 181ms/step - loss: 1.3477 - accuracy: 0.5205 - val_loss: 1.0468 - val_accuracy: 0.6462
Epoch 6/20
90/90 [==============================] - 19s 210ms/step - loss: 1.3500 - accuracy: 0.5256 - val_loss: 0.7521 - val_accuracy: 0.7432
Epoch 7/20
90/90 [==============================] - 17s 191ms/step - loss: 1.2068 - accuracy: 0.5576 - val_loss: 0.8325 - val_accuracy: 0.7

In [ ]:
time.sleep(400)

In [106]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
94/94 [==============================] - 18s 176ms/step - loss: 2.0459 - accuracy: 0.3260 - val_loss: 0.8203 - val_accuracy: 0.7635
Epoch 2/20
94/94 [==============================] - 16s 172ms/step - loss: 1.5887 - accuracy: 0.4197 - val_loss: 0.7366 - val_accuracy: 0.7737
Epoch 3/20
94/94 [==============================] - 16s 175ms/step - loss: 1.4595 - accuracy: 0.4711 - val_loss: 0.7904 - val_accuracy: 0.7563
Epoch 4/20
94/94 [==============================] - 16s 172ms/step - loss: 1.3781 - accuracy: 0.4967 - val_loss: 0.7604 - val_accuracy: 0.7607
Epoch 5/20
94/94 [==============================] - 16s 170ms/step - loss: 1.2857 - accuracy: 0.5301 - val_loss: 0.8003 - val_accuracy: 0.7347
Epoch 6/20
94/94 [==============================] - 16s 171ms/step - loss: 1.2454 - accuracy: 0.5456 - val_loss: 0.7262 - val_accuracy: 0.7602
Epoch 7/20
94/94 [==============================] - 16s 171ms/step - loss: 1.1951 - accuracy: 0.5645 - val_loss: 0.8902 - val_accuracy: 0.7

In [108]:
new_model_dg_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_dg_e2"

i=0

for model in models_dg:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dg_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dg_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dg_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dg_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dg_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dg_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_dg_e2_6\assets
Model has been saved
INFO:tensorfl

In [109]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [110]:

gc.collect()

432286

### Softmax 

In [111]:
#softmax values
se_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/softmax_values.npy"
se_values = np.load(se_direction)[len(x_train):] 
print(len(se_values))



6000


In [112]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_adversary_training)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_adversary_training)

In [113]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  300
300
1 :
0  ->  600
600
2 :
0  ->  900
900
3 :
0  ->  1200
1200
4 :
0  ->  1500
1500
5 :
0  ->  1800
1800
6 :
0  ->  2100
2100
7 :
0  ->  2400
2400
8 :
0  ->  2700
2700
9 :
0  ->  3000
3000
10 :
0  ->  3300
3300
11 :
0  ->  3600
3600
12 :
0  ->  3900
3900
13 :
0  ->  4200
4200
14 :
0  ->  4500
4500
15 :
0  ->  4800
4800
16 :
0  ->  5100
5100
17 :
0  ->  5400
5400
18 :
0  ->  5700
5700
19 :
Last
0  ->  6000
6000


In [114]:
print(model_dir)
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_se.append(model)

C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [115]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
5/5 [==============================] - 8s 2s/step - loss: 2.9722 - accuracy: 0.1939 - val_loss: 1.2527 - val_accuracy: 0.5846
Epoch 2/20
5/5 [==============================] - 6s 1s/step - loss: 1.9358 - accuracy: 0.3061 - val_loss: 1.0746 - val_accuracy: 0.6685
Epoch 3/20
5/5 [==============================] - 6s 1s/step - loss: 1.6505 - accuracy: 0.3727 - val_loss: 1.0421 - val_accuracy: 0.7072
Epoch 4/20
5/5 [==============================] - 6s 1s/step - loss: 1.5621 - accuracy: 0.4331 - val_loss: 0.9208 - val_accuracy: 0.7111
Epoch 5/20
5/5 [==============================] - 6s 2s/step - loss: 1.4259 - accuracy: 0.5340 - val_loss: 0.8781 - val_accuracy: 0.7310
Epoch 6/20
5/5 [==============================] - 7s 2s/step - loss: 1.3425 - accuracy: 0.4921 - val_loss: 0.8146 - val_accuracy: 0.7326
Epoch 7/20
5/5 [==============================] - 7s 2s/step - loss: 1.2451 - accuracy: 0.5846 - val_loss: 0.8220 - val_accuracy: 0.7259
Epoch 8/20
5/5 [=======================

In [116]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
10/10 [==============================] - 12s 1s/step - loss: 2.6677 - accuracy: 0.2081 - val_loss: 1.2529 - val_accuracy: 0.6611
Epoch 2/20
10/10 [==============================] - 9s 984ms/step - loss: 1.9123 - accuracy: 0.3103 - val_loss: 1.0649 - val_accuracy: 0.6826
Epoch 3/20
10/10 [==============================] - 9s 976ms/step - loss: 1.7068 - accuracy: 0.3784 - val_loss: 1.1219 - val_accuracy: 0.7108
Epoch 4/20
10/10 [==============================] - 9s 974ms/step - loss: 1.6314 - accuracy: 0.4370 - val_loss: 0.8879 - val_accuracy: 0.7044
Epoch 5/20
10/10 [==============================] - 9s 976ms/step - loss: 1.5538 - accuracy: 0.4526 - val_loss: 0.9243 - val_accuracy: 0.7371
Epoch 6/20
10/10 [==============================] - 9s 972ms/step - loss: 1.2834 - accuracy: 0.5250 - val_loss: 0.9056 - val_accuracy: 0.7262
Epoch 7/20
10/10 [==============================] - 9s 980ms/step - loss: 1.3181 - accuracy: 0.5355 - val_loss: 0.7964 - val_accuracy: 0.7274
Epoch 

In [117]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
15/15 [==============================] - 13s 755ms/step - loss: 2.4222 - accuracy: 0.2220 - val_loss: 1.1537 - val_accuracy: 0.6344
Epoch 2/20
15/15 [==============================] - 10s 679ms/step - loss: 1.9487 - accuracy: 0.3149 - val_loss: 1.1696 - val_accuracy: 0.6771
Epoch 3/20
15/15 [==============================] - 10s 675ms/step - loss: 1.6890 - accuracy: 0.3935 - val_loss: 1.0947 - val_accuracy: 0.6246
Epoch 4/20
15/15 [==============================] - 10s 665ms/step - loss: 1.6034 - accuracy: 0.4094 - val_loss: 1.1907 - val_accuracy: 0.6109
Epoch 5/20
15/15 [==============================] - 10s 674ms/step - loss: 1.4732 - accuracy: 0.4753 - val_loss: 0.9370 - val_accuracy: 0.6936
Epoch 6/20
15/15 [==============================] - 10s 669ms/step - loss: 1.3909 - accuracy: 0.4567 - val_loss: 1.1456 - val_accuracy: 0.6068
Epoch 7/20
15/15 [==============================] - 10s 672ms/step - loss: 1.3173 - accuracy: 0.5288 - val_loss: 1.0296 - val_accuracy: 0.65

In [118]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
19/19 [==============================] - 12s 593ms/step - loss: 2.5524 - accuracy: 0.2062 - val_loss: 1.2090 - val_accuracy: 0.6980
Epoch 2/20
19/19 [==============================] - 10s 567ms/step - loss: 1.8415 - accuracy: 0.3470 - val_loss: 1.1182 - val_accuracy: 0.6782
Epoch 3/20
19/19 [==============================] - 11s 575ms/step - loss: 1.6975 - accuracy: 0.4130 - val_loss: 0.9161 - val_accuracy: 0.7124
Epoch 4/20
19/19 [==============================] - 10s 572ms/step - loss: 1.6202 - accuracy: 0.4088 - val_loss: 1.1903 - val_accuracy: 0.6260
Epoch 5/20
19/19 [==============================] - 11s 577ms/step - loss: 1.4467 - accuracy: 0.4664 - val_loss: 0.9667 - val_accuracy: 0.6923
Epoch 6/20
19/19 [==============================] - 10s 572ms/step - loss: 1.3245 - accuracy: 0.5126 - val_loss: 0.8798 - val_accuracy: 0.7118
Epoch 7/20
19/19 [==============================] - 10s 566ms/step - loss: 1.2376 - accuracy: 0.5580 - val_loss: 0.9272 - val_accuracy: 0.69

In [119]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
24/24 [==============================] - 13s 496ms/step - loss: 2.7776 - accuracy: 0.2181 - val_loss: 1.2612 - val_accuracy: 0.6879
Epoch 2/20
24/24 [==============================] - 11s 492ms/step - loss: 1.8829 - accuracy: 0.3097 - val_loss: 1.2508 - val_accuracy: 0.6428
Epoch 3/20
24/24 [==============================] - 11s 479ms/step - loss: 1.7036 - accuracy: 0.4030 - val_loss: 1.0075 - val_accuracy: 0.6895
Epoch 4/20
24/24 [==============================] - 12s 527ms/step - loss: 1.5617 - accuracy: 0.4300 - val_loss: 0.9576 - val_accuracy: 0.6879
Epoch 5/20
24/24 [==============================] - 12s 506ms/step - loss: 1.4912 - accuracy: 0.4865 - val_loss: 0.9416 - val_accuracy: 0.6901
Epoch 6/20
24/24 [==============================] - 12s 512ms/step - loss: 1.3346 - accuracy: 0.5393 - val_loss: 0.9579 - val_accuracy: 0.6965
Epoch 7/20
24/24 [==============================] - 12s 494ms/step - loss: 1.1979 - accuracy: 0.6103 - val_loss: 0.9328 - val_accuracy: 0.69

In [120]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
29/29 [==============================] - 14s 452ms/step - loss: 2.4190 - accuracy: 0.2408 - val_loss: 1.1955 - val_accuracy: 0.6659
Epoch 2/20
29/29 [==============================] - 12s 431ms/step - loss: 1.8162 - accuracy: 0.3344 - val_loss: 0.8977 - val_accuracy: 0.7104
Epoch 3/20
29/29 [==============================] - 12s 429ms/step - loss: 1.6881 - accuracy: 0.4138 - val_loss: 0.9266 - val_accuracy: 0.7068
Epoch 4/20
29/29 [==============================] - 12s 429ms/step - loss: 1.5431 - accuracy: 0.4493 - val_loss: 0.8844 - val_accuracy: 0.7075
Epoch 5/20
29/29 [==============================] - 12s 430ms/step - loss: 1.4525 - accuracy: 0.4689 - val_loss: 1.0830 - val_accuracy: 0.6548
Epoch 6/20
29/29 [==============================] - 12s 434ms/step - loss: 1.3828 - accuracy: 0.5114 - val_loss: 0.9868 - val_accuracy: 0.6722
Epoch 7/20
29/29 [==============================] - 13s 465ms/step - loss: 1.2715 - accuracy: 0.5586 - val_loss: 0.9238 - val_accuracy: 0.71

In [121]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
33/33 [==============================] - 15s 423ms/step - loss: 2.5439 - accuracy: 0.2264 - val_loss: 1.2851 - val_accuracy: 0.6301
Epoch 2/20
33/33 [==============================] - 13s 409ms/step - loss: 1.7671 - accuracy: 0.3673 - val_loss: 1.0309 - val_accuracy: 0.6667
Epoch 3/20
33/33 [==============================] - 13s 412ms/step - loss: 1.5856 - accuracy: 0.4262 - val_loss: 1.0213 - val_accuracy: 0.6603
Epoch 4/20
33/33 [==============================] - 13s 407ms/step - loss: 1.5707 - accuracy: 0.4384 - val_loss: 1.0845 - val_accuracy: 0.6635
Epoch 5/20
33/33 [==============================] - 13s 407ms/step - loss: 1.3545 - accuracy: 0.5259 - val_loss: 1.0253 - val_accuracy: 0.6747
Epoch 6/20
33/33 [==============================] - 13s 407ms/step - loss: 1.2693 - accuracy: 0.5567 - val_loss: 0.9520 - val_accuracy: 0.6905
Epoch 7/20
33/33 [==============================] - 13s 412ms/step - loss: 1.1185 - accuracy: 0.6104 - val_loss: 0.9628 - val_accuracy: 0.68

In [122]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
38/38 [==============================] - 16s 381ms/step - loss: 2.4026 - accuracy: 0.2383 - val_loss: 1.2633 - val_accuracy: 0.6022
Epoch 2/20
38/38 [==============================] - 14s 385ms/step - loss: 1.8681 - accuracy: 0.3139 - val_loss: 1.1204 - val_accuracy: 0.6323
Epoch 3/20
38/38 [==============================] - 16s 426ms/step - loss: 1.6723 - accuracy: 0.4065 - val_loss: 0.8960 - val_accuracy: 0.6982
Epoch 4/20
38/38 [==============================] - 16s 415ms/step - loss: 1.5407 - accuracy: 0.4489 - val_loss: 0.9618 - val_accuracy: 0.7036
Epoch 5/20
38/38 [==============================] - 15s 396ms/step - loss: 1.3991 - accuracy: 0.5136 - val_loss: 1.0534 - val_accuracy: 0.6672
Epoch 6/20
38/38 [==============================] - 14s 362ms/step - loss: 1.3516 - accuracy: 0.5313 - val_loss: 0.9329 - val_accuracy: 0.6881
Epoch 7/20
38/38 [==============================] - 13s 360ms/step - loss: 1.2249 - accuracy: 0.5302 - val_loss: 0.9060 - val_accuracy: 0.68

In [123]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
43/43 [==============================] - 17s 362ms/step - loss: 2.3979 - accuracy: 0.2306 - val_loss: 1.1937 - val_accuracy: 0.6232
Epoch 2/20
43/43 [==============================] - 15s 358ms/step - loss: 1.7821 - accuracy: 0.3581 - val_loss: 1.0462 - val_accuracy: 0.6751
Epoch 3/20
43/43 [==============================] - 15s 344ms/step - loss: 1.6327 - accuracy: 0.4058 - val_loss: 1.1805 - val_accuracy: 0.6131
Epoch 4/20
43/43 [==============================] - 14s 338ms/step - loss: 1.4421 - accuracy: 0.4739 - val_loss: 0.9898 - val_accuracy: 0.6643
Epoch 5/20
43/43 [==============================] - 15s 358ms/step - loss: 1.3747 - accuracy: 0.5130 - val_loss: 1.0353 - val_accuracy: 0.6403
Epoch 6/20
43/43 [==============================] - 15s 347ms/step - loss: 1.3031 - accuracy: 0.5375 - val_loss: 1.1437 - val_accuracy: 0.6048
Epoch 7/20
43/43 [==============================] - 14s 339ms/step - loss: 1.2021 - accuracy: 0.5757 - val_loss: 1.0125 - val_accuracy: 0.67

In [124]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
47/47 [==============================] - 18s 361ms/step - loss: 2.4121 - accuracy: 0.2441 - val_loss: 1.2405 - val_accuracy: 0.6038
Epoch 2/20
47/47 [==============================] - 16s 344ms/step - loss: 1.7546 - accuracy: 0.3469 - val_loss: 1.1478 - val_accuracy: 0.6323
Epoch 3/20
47/47 [==============================] - 16s 349ms/step - loss: 1.6308 - accuracy: 0.4065 - val_loss: 1.1172 - val_accuracy: 0.6626
Epoch 4/20
47/47 [==============================] - 16s 351ms/step - loss: 1.4694 - accuracy: 0.4815 - val_loss: 1.1155 - val_accuracy: 0.6338
Epoch 5/20
47/47 [==============================] - 16s 350ms/step - loss: 1.4273 - accuracy: 0.4961 - val_loss: 1.0643 - val_accuracy: 0.6612
Epoch 6/20
47/47 [==============================] - 16s 333ms/step - loss: 1.2614 - accuracy: 0.5437 - val_loss: 0.9985 - val_accuracy: 0.6671
Epoch 7/20
47/47 [==============================] - 15s 332ms/step - loss: 1.1987 - accuracy: 0.5619 - val_loss: 0.9934 - val_accuracy: 0.66

In [125]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
52/52 [==============================] - 20s 365ms/step - loss: 2.3391 - accuracy: 0.2245 - val_loss: 1.3110 - val_accuracy: 0.6201
Epoch 2/20
52/52 [==============================] - 18s 350ms/step - loss: 1.8292 - accuracy: 0.3375 - val_loss: 1.0498 - val_accuracy: 0.6933
Epoch 3/20
52/52 [==============================] - 19s 366ms/step - loss: 1.6437 - accuracy: 0.4083 - val_loss: 0.9911 - val_accuracy: 0.6856
Epoch 4/20
52/52 [==============================] - 17s 326ms/step - loss: 1.5206 - accuracy: 0.4426 - val_loss: 1.1019 - val_accuracy: 0.6454
Epoch 5/20
52/52 [==============================] - 16s 316ms/step - loss: 1.4022 - accuracy: 0.4816 - val_loss: 0.9660 - val_accuracy: 0.6889
Epoch 6/20
52/52 [==============================] - 16s 319ms/step - loss: 1.3393 - accuracy: 0.5149 - val_loss: 0.9876 - val_accuracy: 0.6826
Epoch 7/20
52/52 [==============================] - 16s 311ms/step - loss: 1.2491 - accuracy: 0.5543 - val_loss: 0.9793 - val_accuracy: 0.6

In [126]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
57/57 [==============================] - 22s 366ms/step - loss: 2.2667 - accuracy: 0.2407 - val_loss: 1.2095 - val_accuracy: 0.6496
Epoch 2/20
57/57 [==============================] - 20s 360ms/step - loss: 1.7021 - accuracy: 0.3733 - val_loss: 1.1549 - val_accuracy: 0.6423
Epoch 3/20
57/57 [==============================] - 19s 330ms/step - loss: 1.5684 - accuracy: 0.4289 - val_loss: 1.0913 - val_accuracy: 0.6568
Epoch 4/20
57/57 [==============================] - 19s 332ms/step - loss: 1.4258 - accuracy: 0.4764 - val_loss: 1.0335 - val_accuracy: 0.6349
Epoch 5/20
57/57 [==============================] - 19s 342ms/step - loss: 1.3522 - accuracy: 0.5106 - val_loss: 1.1406 - val_accuracy: 0.6193
Epoch 6/20
57/57 [==============================] - 17s 302ms/step - loss: 1.2653 - accuracy: 0.5367 - val_loss: 1.0248 - val_accuracy: 0.6492
Epoch 7/20
57/57 [==============================] - 19s 328ms/step - loss: 1.1553 - accuracy: 0.5721 - val_loss: 0.9130 - val_accuracy: 0.6

In [127]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
61/61 [==============================] - 22s 341ms/step - loss: 2.3989 - accuracy: 0.2126 - val_loss: 1.1745 - val_accuracy: 0.6532
Epoch 2/20
61/61 [==============================] - 18s 294ms/step - loss: 1.7189 - accuracy: 0.3603 - val_loss: 1.0265 - val_accuracy: 0.6778
Epoch 3/20
61/61 [==============================] - 17s 283ms/step - loss: 1.5509 - accuracy: 0.4325 - val_loss: 1.0818 - val_accuracy: 0.6804
Epoch 4/20
61/61 [==============================] - 20s 323ms/step - loss: 1.5181 - accuracy: 0.4595 - val_loss: 1.0046 - val_accuracy: 0.6827
Epoch 5/20
61/61 [==============================] - 18s 296ms/step - loss: 1.3731 - accuracy: 0.4990 - val_loss: 0.9832 - val_accuracy: 0.6862
Epoch 6/20
61/61 [==============================] - 18s 290ms/step - loss: 1.2985 - accuracy: 0.5406 - val_loss: 0.9519 - val_accuracy: 0.6899
Epoch 7/20
61/61 [==============================] - 18s 296ms/step - loss: 1.2050 - accuracy: 0.5643 - val_loss: 0.9762 - val_accuracy: 0.6

In [128]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
66/66 [==============================] - 20s 285ms/step - loss: 2.2857 - accuracy: 0.2276 - val_loss: 0.8674 - val_accuracy: 0.7537
Epoch 2/20
66/66 [==============================] - 18s 281ms/step - loss: 1.7416 - accuracy: 0.3558 - val_loss: 1.0003 - val_accuracy: 0.6834
Epoch 3/20
66/66 [==============================] - 18s 281ms/step - loss: 1.6276 - accuracy: 0.4024 - val_loss: 1.0055 - val_accuracy: 0.7039
Epoch 4/20
66/66 [==============================] - 19s 283ms/step - loss: 1.4508 - accuracy: 0.4701 - val_loss: 1.0327 - val_accuracy: 0.6913
Epoch 5/20
66/66 [==============================] - 18s 280ms/step - loss: 1.4379 - accuracy: 0.4858 - val_loss: 0.9835 - val_accuracy: 0.6818
Epoch 6/20
66/66 [==============================] - 18s 279ms/step - loss: 1.3259 - accuracy: 0.5062 - val_loss: 0.8218 - val_accuracy: 0.7278
Epoch 7/20
66/66 [==============================] - 18s 281ms/step - loss: 1.2654 - accuracy: 0.5399 - val_loss: 0.7721 - val_accuracy: 0.7

In [129]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
71/71 [==============================] - 21s 277ms/step - loss: 2.2458 - accuracy: 0.2645 - val_loss: 0.8698 - val_accuracy: 0.7553
Epoch 2/20
71/71 [==============================] - 19s 268ms/step - loss: 1.7370 - accuracy: 0.3611 - val_loss: 0.8624 - val_accuracy: 0.7106
Epoch 3/20
71/71 [==============================] - 19s 273ms/step - loss: 1.6011 - accuracy: 0.4253 - val_loss: 0.9000 - val_accuracy: 0.7060
Epoch 4/20
71/71 [==============================] - 20s 280ms/step - loss: 1.5013 - accuracy: 0.4602 - val_loss: 0.8616 - val_accuracy: 0.7127
Epoch 5/20
71/71 [==============================] - 20s 279ms/step - loss: 1.3985 - accuracy: 0.4981 - val_loss: 1.1198 - val_accuracy: 0.6503
Epoch 6/20
71/71 [==============================] - 20s 278ms/step - loss: 1.3106 - accuracy: 0.5038 - val_loss: 0.8325 - val_accuracy: 0.7096
Epoch 7/20
71/71 [==============================] - 19s 271ms/step - loss: 1.2527 - accuracy: 0.5619 - val_loss: 0.9339 - val_accuracy: 0.6

In [130]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
75/75 [==============================] - 22s 271ms/step - loss: 2.1965 - accuracy: 0.2776 - val_loss: 0.9913 - val_accuracy: 0.6852
Epoch 2/20
75/75 [==============================] - 20s 268ms/step - loss: 1.7346 - accuracy: 0.3611 - val_loss: 0.9058 - val_accuracy: 0.7329
Epoch 3/20
75/75 [==============================] - 20s 269ms/step - loss: 1.5526 - accuracy: 0.4317 - val_loss: 0.8970 - val_accuracy: 0.7218
Epoch 4/20
75/75 [==============================] - 20s 265ms/step - loss: 1.4665 - accuracy: 0.4709 - val_loss: 0.8723 - val_accuracy: 0.7221
Epoch 5/20
75/75 [==============================] - 20s 272ms/step - loss: 1.3639 - accuracy: 0.5076 - val_loss: 0.8122 - val_accuracy: 0.7434
Epoch 6/20
75/75 [==============================] - 20s 265ms/step - loss: 1.3180 - accuracy: 0.5255 - val_loss: 0.8149 - val_accuracy: 0.7363
Epoch 7/20
75/75 [==============================] - 20s 266ms/step - loss: 1.2550 - accuracy: 0.5507 - val_loss: 0.7865 - val_accuracy: 0.7

In [131]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
80/80 [==============================] - 24s 279ms/step - loss: 2.2079 - accuracy: 0.2774 - val_loss: 0.8058 - val_accuracy: 0.7491
Epoch 2/20
80/80 [==============================] - 22s 282ms/step - loss: 1.6917 - accuracy: 0.3712 - val_loss: 0.8957 - val_accuracy: 0.7285
Epoch 3/20
80/80 [==============================] - 22s 276ms/step - loss: 1.5942 - accuracy: 0.4213 - val_loss: 0.8570 - val_accuracy: 0.7253
Epoch 4/20
80/80 [==============================] - 21s 267ms/step - loss: 1.4720 - accuracy: 0.4804 - val_loss: 0.8145 - val_accuracy: 0.7460
Epoch 5/20
80/80 [==============================] - 22s 278ms/step - loss: 1.3749 - accuracy: 0.5060 - val_loss: 0.8058 - val_accuracy: 0.7238
Epoch 6/20
80/80 [==============================] - 21s 261ms/step - loss: 1.3091 - accuracy: 0.5311 - val_loss: 0.8268 - val_accuracy: 0.7338
Epoch 7/20
80/80 [==============================] - 21s 260ms/step - loss: 1.2577 - accuracy: 0.5603 - val_loss: 0.7764 - val_accuracy: 0.7

In [132]:
time.sleep(400)

In [133]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
85/85 [==============================] - 16s 170ms/step - loss: 2.3026 - accuracy: 0.2801 - val_loss: 0.9587 - val_accuracy: 0.7431
Epoch 2/20
85/85 [==============================] - 14s 168ms/step - loss: 1.6941 - accuracy: 0.3803 - val_loss: 0.8414 - val_accuracy: 0.7336
Epoch 3/20
85/85 [==============================] - 14s 170ms/step - loss: 1.5576 - accuracy: 0.4306 - val_loss: 0.9562 - val_accuracy: 0.7038
Epoch 4/20
85/85 [==============================] - 15s 177ms/step - loss: 1.4710 - accuracy: 0.4600 - val_loss: 0.7836 - val_accuracy: 0.7437
Epoch 5/20
85/85 [==============================] - 17s 198ms/step - loss: 1.3686 - accuracy: 0.5088 - val_loss: 1.0372 - val_accuracy: 0.6688
Epoch 6/20
85/85 [==============================] - 18s 217ms/step - loss: 1.2814 - accuracy: 0.5290 - val_loss: 0.7349 - val_accuracy: 0.7571
Epoch 7/20
85/85 [==============================] - 21s 251ms/step - loss: 1.2387 - accuracy: 0.5566 - val_loss: 0.9265 - val_accuracy: 0.6

In [ ]:
time.sleep(400)

In [134]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
90/90 [==============================] - 24s 255ms/step - loss: 2.0708 - accuracy: 0.3135 - val_loss: 1.1390 - val_accuracy: 0.6308
Epoch 2/20
90/90 [==============================] - 23s 253ms/step - loss: 1.6705 - accuracy: 0.3950 - val_loss: 1.0525 - val_accuracy: 0.6784
Epoch 3/20
90/90 [==============================] - 23s 254ms/step - loss: 1.5320 - accuracy: 0.4459 - val_loss: 1.1436 - val_accuracy: 0.5568
Epoch 4/20
90/90 [==============================] - 23s 254ms/step - loss: 1.4453 - accuracy: 0.4803 - val_loss: 0.7740 - val_accuracy: 0.7475
Epoch 5/20
90/90 [==============================] - 23s 252ms/step - loss: 1.3266 - accuracy: 0.5103 - val_loss: 0.7832 - val_accuracy: 0.7361
Epoch 6/20
90/90 [==============================] - 23s 259ms/step - loss: 1.2577 - accuracy: 0.5409 - val_loss: 0.9278 - val_accuracy: 0.6849
Epoch 7/20
90/90 [==============================] - 23s 255ms/step - loss: 1.2368 - accuracy: 0.5585 - val_loss: 0.7151 - val_accuracy: 0.7

In [ ]:
time.sleep(400)

In [135]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
94/94 [==============================] - 27s 271ms/step - loss: 2.1449 - accuracy: 0.3310 - val_loss: 0.8886 - val_accuracy: 0.7514
Epoch 2/20
94/94 [==============================] - 24s 260ms/step - loss: 1.5675 - accuracy: 0.4170 - val_loss: 0.9050 - val_accuracy: 0.7398
Epoch 3/20
94/94 [==============================] - 25s 263ms/step - loss: 1.4724 - accuracy: 0.4553 - val_loss: 0.6718 - val_accuracy: 0.7814
Epoch 4/20
94/94 [==============================] - 25s 262ms/step - loss: 1.3883 - accuracy: 0.4924 - val_loss: 0.7439 - val_accuracy: 0.7633
Epoch 5/20
94/94 [==============================] - 24s 258ms/step - loss: 1.3507 - accuracy: 0.5213 - val_loss: 0.7500 - val_accuracy: 0.7492
Epoch 6/20
94/94 [==============================] - 25s 264ms/step - loss: 1.2423 - accuracy: 0.5501 - val_loss: 0.8431 - val_accuracy: 0.7287
Epoch 7/20
94/94 [==============================] - 25s 265ms/step - loss: 1.1830 - accuracy: 0.5610 - val_loss: 0.7434 - val_accuracy: 0.7

In [ ]:
len(models_se)

In [136]:
new_model_se_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_se_e2"

i=0

for model in models_se:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_se_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_se_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_se_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_se_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_se_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_se_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_se_e2_6\assets
Model has been saved
INFO:tensorfl

In [137]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [138]:
gc.collect()

548934

## Training guided by Random values

In [139]:
save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)[len(x_train):] 
print(len(random_indexes))


6000


In [140]:
len(random_indexes)

6000

In [141]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_adversary_training)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_adversary_training)

In [142]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  300
300
1 :
0  ->  600
600
2 :
0  ->  900
900
3 :
0  ->  1200
1200
4 :
0  ->  1500
1500
5 :
0  ->  1800
1800
6 :
0  ->  2100
2100
7 :
0  ->  2400
2400
8 :
0  ->  2700
2700
9 :
0  ->  3000
3000
10 :
0  ->  3300
3300
11 :
0  ->  3600
3600
12 :
0  ->  3900
3900
13 :
0  ->  4200
4200
14 :
0  ->  4500
4500
15 :
0  ->  4800
4800
16 :
0  ->  5100
5100
17 :
0  ->  5400
5400
18 :
0  ->  5700
5700
19 :
Last
0  ->  6000
6000


In [143]:
len(image_sets_random)

20

In [144]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_random.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [145]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
5/5 [==============================] - 11s 2s/step - loss: 3.7131 - accuracy: 0.2352 - val_loss: 1.1480 - val_accuracy: 0.7212
Epoch 2/20
5/5 [==============================] - 9s 2s/step - loss: 1.9685 - accuracy: 0.3094 - val_loss: 1.0332 - val_accuracy: 0.7366
Epoch 3/20
5/5 [==============================] - 9s 2s/step - loss: 1.8358 - accuracy: 0.3395 - val_loss: 0.9231 - val_accuracy: 0.7677
Epoch 4/20
5/5 [==============================] - 9s 2s/step - loss: 1.7282 - accuracy: 0.3652 - val_loss: 0.8918 - val_accuracy: 0.7759
Epoch 5/20
5/5 [==============================] - 9s 2s/step - loss: 1.6286 - accuracy: 0.3935 - val_loss: 0.8280 - val_accuracy: 0.7400
Epoch 6/20
5/5 [==============================] - 9s 2s/step - loss: 1.4850 - accuracy: 0.4016 - val_loss: 0.9058 - val_accuracy: 0.7200
Epoch 7/20
5/5 [==============================] - 9s 2s/step - loss: 1.3822 - accuracy: 0.4694 - val_loss: 0.8400 - val_accuracy: 0.7433
Epoch 8/20
5/5 [======================

In [146]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
10/10 [==============================] - 12s 1s/step - loss: 3.2984 - accuracy: 0.2588 - val_loss: 1.0496 - val_accuracy: 0.7543
Epoch 2/20
10/10 [==============================] - 10s 1s/step - loss: 1.8053 - accuracy: 0.3433 - val_loss: 1.1336 - val_accuracy: 0.6973
Epoch 3/20
10/10 [==============================] - 10s 1s/step - loss: 1.6954 - accuracy: 0.3625 - val_loss: 0.8180 - val_accuracy: 0.7703
Epoch 4/20
10/10 [==============================] - 10s 1s/step - loss: 1.6543 - accuracy: 0.3834 - val_loss: 0.8370 - val_accuracy: 0.7319
Epoch 5/20
10/10 [==============================] - 10s 1s/step - loss: 1.5736 - accuracy: 0.3964 - val_loss: 0.8712 - val_accuracy: 0.7443
Epoch 6/20
10/10 [==============================] - 10s 1s/step - loss: 1.5044 - accuracy: 0.4293 - val_loss: 0.8460 - val_accuracy: 0.7275
Epoch 7/20
10/10 [==============================] - 10s 1s/step - loss: 1.4056 - accuracy: 0.4772 - val_loss: 0.7854 - val_accuracy: 0.7433
Epoch 8/20
10/10 [

In [147]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
15/15 [==============================] - 13s 823ms/step - loss: 2.7635 - accuracy: 0.2897 - val_loss: 0.9877 - val_accuracy: 0.7380
Epoch 2/20
15/15 [==============================] - 11s 795ms/step - loss: 1.8429 - accuracy: 0.3406 - val_loss: 0.9892 - val_accuracy: 0.7023
Epoch 3/20
15/15 [==============================] - 11s 792ms/step - loss: 1.6387 - accuracy: 0.3785 - val_loss: 1.0981 - val_accuracy: 0.6683
Epoch 4/20
15/15 [==============================] - 11s 785ms/step - loss: 1.6386 - accuracy: 0.3888 - val_loss: 0.9075 - val_accuracy: 0.6999
Epoch 5/20
15/15 [==============================] - 11s 786ms/step - loss: 1.6757 - accuracy: 0.4173 - val_loss: 1.0081 - val_accuracy: 0.6744
Epoch 6/20
15/15 [==============================] - 11s 786ms/step - loss: 1.4615 - accuracy: 0.4404 - val_loss: 0.9724 - val_accuracy: 0.7017
Epoch 7/20
15/15 [==============================] - 11s 786ms/step - loss: 1.4568 - accuracy: 0.4681 - val_loss: 0.9386 - val_accuracy: 0.71

In [148]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
19/19 [==============================] - 14s 675ms/step - loss: 2.8006 - accuracy: 0.2867 - val_loss: 0.9739 - val_accuracy: 0.7677
Epoch 2/20
19/19 [==============================] - 12s 663ms/step - loss: 1.6983 - accuracy: 0.3967 - val_loss: 0.8976 - val_accuracy: 0.7379
Epoch 3/20
19/19 [==============================] - 12s 659ms/step - loss: 1.6735 - accuracy: 0.3908 - val_loss: 0.8392 - val_accuracy: 0.7347
Epoch 4/20
19/19 [==============================] - 12s 655ms/step - loss: 1.5078 - accuracy: 0.4331 - val_loss: 0.8416 - val_accuracy: 0.7563
Epoch 5/20
19/19 [==============================] - 12s 656ms/step - loss: 1.5320 - accuracy: 0.4511 - val_loss: 0.7525 - val_accuracy: 0.7726
Epoch 6/20
19/19 [==============================] - 12s 658ms/step - loss: 1.3945 - accuracy: 0.4873 - val_loss: 0.8166 - val_accuracy: 0.7426
Epoch 7/20
19/19 [==============================] - 12s 665ms/step - loss: 1.3718 - accuracy: 0.4914 - val_loss: 0.7840 - val_accuracy: 0.75

In [149]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
24/24 [==============================] - 16s 611ms/step - loss: 2.5388 - accuracy: 0.3275 - val_loss: 1.0365 - val_accuracy: 0.7028
Epoch 2/20
24/24 [==============================] - 14s 583ms/step - loss: 1.7732 - accuracy: 0.3536 - val_loss: 0.9717 - val_accuracy: 0.7362
Epoch 3/20
24/24 [==============================] - 13s 571ms/step - loss: 1.6525 - accuracy: 0.3907 - val_loss: 1.0167 - val_accuracy: 0.7222
Epoch 4/20
24/24 [==============================] - 14s 591ms/step - loss: 1.5987 - accuracy: 0.4175 - val_loss: 0.7987 - val_accuracy: 0.7420
Epoch 5/20
24/24 [==============================] - 14s 607ms/step - loss: 1.5638 - accuracy: 0.4211 - val_loss: 0.8340 - val_accuracy: 0.7318
Epoch 6/20
24/24 [==============================] - 14s 580ms/step - loss: 1.4765 - accuracy: 0.4705 - val_loss: 0.7258 - val_accuracy: 0.7673
Epoch 7/20
24/24 [==============================] - 14s 592ms/step - loss: 1.3361 - accuracy: 0.4937 - val_loss: 0.7919 - val_accuracy: 0.74

In [150]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
29/29 [==============================] - 16s 521ms/step - loss: 2.5488 - accuracy: 0.3019 - val_loss: 0.8270 - val_accuracy: 0.7557
Epoch 2/20
29/29 [==============================] - 15s 528ms/step - loss: 1.7060 - accuracy: 0.3747 - val_loss: 0.9072 - val_accuracy: 0.6998
Epoch 3/20
29/29 [==============================] - 14s 509ms/step - loss: 1.6178 - accuracy: 0.4108 - val_loss: 0.7579 - val_accuracy: 0.7617
Epoch 4/20
29/29 [==============================] - 15s 513ms/step - loss: 1.5285 - accuracy: 0.4457 - val_loss: 0.9544 - val_accuracy: 0.6898
Epoch 5/20
29/29 [==============================] - 14s 509ms/step - loss: 1.4924 - accuracy: 0.4591 - val_loss: 0.8444 - val_accuracy: 0.7486
Epoch 6/20
29/29 [==============================] - 15s 514ms/step - loss: 1.4033 - accuracy: 0.4714 - val_loss: 0.6596 - val_accuracy: 0.7893
Epoch 7/20
29/29 [==============================] - 14s 504ms/step - loss: 1.3364 - accuracy: 0.5149 - val_loss: 0.9644 - val_accuracy: 0.68

In [151]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
33/33 [==============================] - 17s 482ms/step - loss: 2.4118 - accuracy: 0.3162 - val_loss: 0.8997 - val_accuracy: 0.7569
Epoch 2/20
33/33 [==============================] - 15s 473ms/step - loss: 1.7074 - accuracy: 0.3579 - val_loss: 0.9218 - val_accuracy: 0.7137
Epoch 3/20
33/33 [==============================] - 16s 479ms/step - loss: 1.5896 - accuracy: 0.3963 - val_loss: 0.7722 - val_accuracy: 0.7563
Epoch 4/20
33/33 [==============================] - 16s 488ms/step - loss: 1.5471 - accuracy: 0.4319 - val_loss: 0.7659 - val_accuracy: 0.7619
Epoch 5/20
33/33 [==============================] - 15s 475ms/step - loss: 1.4342 - accuracy: 0.4682 - val_loss: 0.8625 - val_accuracy: 0.7301
Epoch 6/20
33/33 [==============================] - 15s 472ms/step - loss: 1.3733 - accuracy: 0.5081 - val_loss: 0.8378 - val_accuracy: 0.7261
Epoch 7/20
33/33 [==============================] - 16s 506ms/step - loss: 1.3225 - accuracy: 0.5100 - val_loss: 0.7572 - val_accuracy: 0.75

In [152]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
38/38 [==============================] - 18s 449ms/step - loss: 2.4199 - accuracy: 0.3049 - val_loss: 0.8433 - val_accuracy: 0.7423
Epoch 2/20
38/38 [==============================] - 17s 442ms/step - loss: 1.6660 - accuracy: 0.3715 - val_loss: 1.0732 - val_accuracy: 0.7018
Epoch 3/20
38/38 [==============================] - 17s 447ms/step - loss: 1.6018 - accuracy: 0.4160 - val_loss: 0.7593 - val_accuracy: 0.7732
Epoch 4/20
38/38 [==============================] - 17s 453ms/step - loss: 1.4904 - accuracy: 0.4426 - val_loss: 0.7708 - val_accuracy: 0.7609
Epoch 5/20
38/38 [==============================] - 17s 443ms/step - loss: 1.4340 - accuracy: 0.4792 - val_loss: 0.7022 - val_accuracy: 0.7667
Epoch 6/20
38/38 [==============================] - 16s 433ms/step - loss: 1.4092 - accuracy: 0.4653 - val_loss: 0.8481 - val_accuracy: 0.7350
Epoch 7/20
38/38 [==============================] - 16s 440ms/step - loss: 1.3339 - accuracy: 0.5267 - val_loss: 0.6891 - val_accuracy: 0.77

In [153]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
43/43 [==============================] - 21s 449ms/step - loss: 2.3647 - accuracy: 0.2990 - val_loss: 1.1221 - val_accuracy: 0.6337
Epoch 2/20
43/43 [==============================] - 19s 446ms/step - loss: 1.6789 - accuracy: 0.3764 - val_loss: 0.8939 - val_accuracy: 0.7278
Epoch 3/20
43/43 [==============================] - 19s 445ms/step - loss: 1.5657 - accuracy: 0.4283 - val_loss: 0.9304 - val_accuracy: 0.7100
Epoch 4/20
43/43 [==============================] - 19s 443ms/step - loss: 1.4442 - accuracy: 0.4713 - val_loss: 0.7519 - val_accuracy: 0.7653
Epoch 5/20
43/43 [==============================] - 19s 442ms/step - loss: 1.4464 - accuracy: 0.4643 - val_loss: 0.8763 - val_accuracy: 0.7133
Epoch 6/20
43/43 [==============================] - 19s 440ms/step - loss: 1.3740 - accuracy: 0.4898 - val_loss: 0.8440 - val_accuracy: 0.7197
Epoch 7/20
43/43 [==============================] - 19s 456ms/step - loss: 1.2796 - accuracy: 0.5291 - val_loss: 0.8566 - val_accuracy: 0.72

In [154]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
47/47 [==============================] - 22s 436ms/step - loss: 2.2614 - accuracy: 0.3087 - val_loss: 0.8545 - val_accuracy: 0.7722
Epoch 2/20
47/47 [==============================] - 21s 459ms/step - loss: 1.6661 - accuracy: 0.3888 - val_loss: 0.7866 - val_accuracy: 0.7754
Epoch 3/20
47/47 [==============================] - 20s 437ms/step - loss: 1.5918 - accuracy: 0.4210 - val_loss: 0.7280 - val_accuracy: 0.7771
Epoch 4/20
47/47 [==============================] - 20s 429ms/step - loss: 1.4688 - accuracy: 0.4579 - val_loss: 0.8080 - val_accuracy: 0.7606
Epoch 5/20
47/47 [==============================] - 21s 444ms/step - loss: 1.4606 - accuracy: 0.4774 - val_loss: 0.7827 - val_accuracy: 0.7524
Epoch 6/20
47/47 [==============================] - 20s 427ms/step - loss: 1.4182 - accuracy: 0.4600 - val_loss: 0.6945 - val_accuracy: 0.7721
Epoch 7/20
47/47 [==============================] - 21s 449ms/step - loss: 1.3072 - accuracy: 0.5233 - val_loss: 0.9430 - val_accuracy: 0.70

In [155]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
52/52 [==============================] - 25s 444ms/step - loss: 2.1480 - accuracy: 0.3274 - val_loss: 1.0315 - val_accuracy: 0.6934
Epoch 2/20
52/52 [==============================] - 23s 439ms/step - loss: 1.6386 - accuracy: 0.3951 - val_loss: 1.1993 - val_accuracy: 0.6171
Epoch 3/20
52/52 [==============================] - 21s 415ms/step - loss: 1.6084 - accuracy: 0.4040 - val_loss: 0.8049 - val_accuracy: 0.7521
Epoch 4/20
52/52 [==============================] - 22s 418ms/step - loss: 1.4643 - accuracy: 0.4393 - val_loss: 0.7853 - val_accuracy: 0.7517
Epoch 5/20
52/52 [==============================] - 21s 402ms/step - loss: 1.4121 - accuracy: 0.4958 - val_loss: 0.8337 - val_accuracy: 0.7207
Epoch 6/20
52/52 [==============================] - 21s 400ms/step - loss: 1.3821 - accuracy: 0.4941 - val_loss: 0.8734 - val_accuracy: 0.7049
Epoch 7/20
52/52 [==============================] - 22s 422ms/step - loss: 1.2800 - accuracy: 0.5397 - val_loss: 0.7255 - val_accuracy: 0.7

In [156]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
57/57 [==============================] - 28s 458ms/step - loss: 2.2832 - accuracy: 0.3161 - val_loss: 1.0510 - val_accuracy: 0.7098
Epoch 2/20
57/57 [==============================] - 24s 425ms/step - loss: 1.6537 - accuracy: 0.3918 - val_loss: 0.9044 - val_accuracy: 0.7197
Epoch 3/20
57/57 [==============================] - 23s 405ms/step - loss: 1.5524 - accuracy: 0.4315 - val_loss: 0.9479 - val_accuracy: 0.6948
Epoch 4/20
57/57 [==============================] - 23s 405ms/step - loss: 1.5042 - accuracy: 0.4640 - val_loss: 0.8074 - val_accuracy: 0.7417
Epoch 5/20
57/57 [==============================] - 23s 405ms/step - loss: 1.4279 - accuracy: 0.4902 - val_loss: 0.7951 - val_accuracy: 0.7456
Epoch 6/20
57/57 [==============================] - 24s 416ms/step - loss: 1.3718 - accuracy: 0.4904 - val_loss: 0.9542 - val_accuracy: 0.7132
Epoch 7/20
57/57 [==============================] - 22s 386ms/step - loss: 1.2765 - accuracy: 0.5228 - val_loss: 0.7933 - val_accuracy: 0.7

In [157]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
61/61 [==============================] - 22s 340ms/step - loss: 2.1296 - accuracy: 0.3237 - val_loss: 0.9078 - val_accuracy: 0.7366
Epoch 2/20
61/61 [==============================] - 20s 334ms/step - loss: 1.6084 - accuracy: 0.4131 - val_loss: 0.8270 - val_accuracy: 0.7752
Epoch 3/20
61/61 [==============================] - 20s 334ms/step - loss: 1.4888 - accuracy: 0.4471 - val_loss: 0.8469 - val_accuracy: 0.7389
Epoch 4/20
61/61 [==============================] - 20s 338ms/step - loss: 1.4227 - accuracy: 0.4844 - val_loss: 0.7783 - val_accuracy: 0.7668
Epoch 5/20
61/61 [==============================] - 20s 334ms/step - loss: 1.3693 - accuracy: 0.4947 - val_loss: 0.7225 - val_accuracy: 0.7717
Epoch 6/20
61/61 [==============================] - 20s 335ms/step - loss: 1.3417 - accuracy: 0.5054 - val_loss: 0.7971 - val_accuracy: 0.7517
Epoch 7/20
61/61 [==============================] - 20s 334ms/step - loss: 1.2411 - accuracy: 0.5530 - val_loss: 0.8863 - val_accuracy: 0.7

In [158]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
66/66 [==============================] - 23s 329ms/step - loss: 2.1290 - accuracy: 0.3394 - val_loss: 1.1179 - val_accuracy: 0.7106
Epoch 2/20
66/66 [==============================] - 22s 329ms/step - loss: 1.6159 - accuracy: 0.4028 - val_loss: 0.8394 - val_accuracy: 0.7582
Epoch 3/20
66/66 [==============================] - 21s 326ms/step - loss: 1.5242 - accuracy: 0.4410 - val_loss: 0.9343 - val_accuracy: 0.7081
Epoch 4/20
66/66 [==============================] - 21s 324ms/step - loss: 1.4268 - accuracy: 0.4759 - val_loss: 0.6864 - val_accuracy: 0.7750
Epoch 5/20
66/66 [==============================] - 21s 326ms/step - loss: 1.3919 - accuracy: 0.5119 - val_loss: 0.8323 - val_accuracy: 0.7316
Epoch 6/20
66/66 [==============================] - 21s 326ms/step - loss: 1.3051 - accuracy: 0.5257 - val_loss: 0.7660 - val_accuracy: 0.7407
Epoch 7/20
66/66 [==============================] - 21s 327ms/step - loss: 1.2042 - accuracy: 0.5715 - val_loss: 0.7800 - val_accuracy: 0.7

In [159]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
71/71 [==============================] - 26s 345ms/step - loss: 2.1092 - accuracy: 0.3129 - val_loss: 0.8288 - val_accuracy: 0.7419
Epoch 2/20
71/71 [==============================] - 24s 340ms/step - loss: 1.6327 - accuracy: 0.4056 - val_loss: 1.0132 - val_accuracy: 0.6973
Epoch 3/20
71/71 [==============================] - 24s 343ms/step - loss: 1.5078 - accuracy: 0.4410 - val_loss: 0.9570 - val_accuracy: 0.6768
Epoch 4/20
71/71 [==============================] - 24s 339ms/step - loss: 1.4578 - accuracy: 0.4771 - val_loss: 0.8807 - val_accuracy: 0.7253
Epoch 5/20
71/71 [==============================] - 24s 345ms/step - loss: 1.3537 - accuracy: 0.5113 - val_loss: 0.6866 - val_accuracy: 0.7658
Epoch 6/20
71/71 [==============================] - 24s 341ms/step - loss: 1.3005 - accuracy: 0.5258 - val_loss: 0.6809 - val_accuracy: 0.7715
Epoch 7/20
71/71 [==============================] - 24s 340ms/step - loss: 1.2358 - accuracy: 0.5481 - val_loss: 0.7884 - val_accuracy: 0.7

In [160]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
75/75 [==============================] - 27s 340ms/step - loss: 2.2224 - accuracy: 0.3058 - val_loss: 0.8783 - val_accuracy: 0.7510
Epoch 2/20
75/75 [==============================] - 25s 330ms/step - loss: 1.5682 - accuracy: 0.4199 - val_loss: 0.8456 - val_accuracy: 0.7552
Epoch 3/20
75/75 [==============================] - 25s 334ms/step - loss: 1.4940 - accuracy: 0.4526 - val_loss: 0.7318 - val_accuracy: 0.7673
Epoch 4/20
75/75 [==============================] - 25s 332ms/step - loss: 1.4001 - accuracy: 0.4906 - val_loss: 0.9084 - val_accuracy: 0.7230
Epoch 5/20
75/75 [==============================] - 25s 331ms/step - loss: 1.3630 - accuracy: 0.5096 - val_loss: 0.7586 - val_accuracy: 0.7487
Epoch 6/20
75/75 [==============================] - 25s 331ms/step - loss: 1.2871 - accuracy: 0.5236 - val_loss: 1.0157 - val_accuracy: 0.6664
Epoch 7/20
75/75 [==============================] - 25s 331ms/step - loss: 1.2633 - accuracy: 0.5445 - val_loss: 0.7338 - val_accuracy: 0.7

In [161]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
80/80 [==============================] - 28s 335ms/step - loss: 2.1923 - accuracy: 0.3002 - val_loss: 0.9279 - val_accuracy: 0.7375
Epoch 2/20
80/80 [==============================] - 26s 330ms/step - loss: 1.5905 - accuracy: 0.4146 - val_loss: 0.9908 - val_accuracy: 0.6785
Epoch 3/20
80/80 [==============================] - 26s 328ms/step - loss: 1.4934 - accuracy: 0.4572 - val_loss: 0.7291 - val_accuracy: 0.7678
Epoch 4/20
80/80 [==============================] - 26s 330ms/step - loss: 1.4342 - accuracy: 0.4750 - val_loss: 0.7898 - val_accuracy: 0.7524
Epoch 5/20
80/80 [==============================] - 26s 329ms/step - loss: 1.3452 - accuracy: 0.5108 - val_loss: 0.8848 - val_accuracy: 0.7330
Epoch 6/20
80/80 [==============================] - 27s 334ms/step - loss: 1.2689 - accuracy: 0.5294 - val_loss: 0.7447 - val_accuracy: 0.7599
Epoch 7/20
80/80 [==============================] - 26s 331ms/step - loss: 1.2043 - accuracy: 0.5600 - val_loss: 0.8909 - val_accuracy: 0.7

In [162]:
time.sleep(400)

In [163]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
85/85 [==============================] - 20s 223ms/step - loss: 2.0754 - accuracy: 0.3335 - val_loss: 0.7028 - val_accuracy: 0.7891
Epoch 2/20
85/85 [==============================] - 19s 223ms/step - loss: 1.5582 - accuracy: 0.4295 - val_loss: 1.0371 - val_accuracy: 0.6751
Epoch 3/20
85/85 [==============================] - 19s 223ms/step - loss: 1.4957 - accuracy: 0.4470 - val_loss: 0.8186 - val_accuracy: 0.7223
Epoch 4/20
85/85 [==============================] - 21s 251ms/step - loss: 1.4198 - accuracy: 0.4876 - val_loss: 0.7356 - val_accuracy: 0.7520
Epoch 5/20
85/85 [==============================] - 25s 291ms/step - loss: 1.3135 - accuracy: 0.5173 - val_loss: 0.7366 - val_accuracy: 0.7511
Epoch 6/20
85/85 [==============================] - 28s 330ms/step - loss: 1.2670 - accuracy: 0.5372 - val_loss: 0.7332 - val_accuracy: 0.7520
Epoch 7/20
85/85 [==============================] - 28s 329ms/step - loss: 1.2065 - accuracy: 0.5736 - val_loss: 0.8126 - val_accuracy: 0.7

In [ ]:
time.sleep(400)

In [164]:
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
90/90 [==============================] - 31s 323ms/step - loss: 2.0952 - accuracy: 0.3305 - val_loss: 1.1100 - val_accuracy: 0.6377
Epoch 2/20
90/90 [==============================] - 29s 320ms/step - loss: 1.5453 - accuracy: 0.4365 - val_loss: 0.8431 - val_accuracy: 0.7386
Epoch 3/20
90/90 [==============================] - 29s 318ms/step - loss: 1.4718 - accuracy: 0.4693 - val_loss: 0.9669 - val_accuracy: 0.6746
Epoch 4/20
90/90 [==============================] - 29s 318ms/step - loss: 1.4141 - accuracy: 0.4931 - val_loss: 0.8963 - val_accuracy: 0.7014
Epoch 5/20
90/90 [==============================] - 29s 319ms/step - loss: 1.3329 - accuracy: 0.5126 - val_loss: 0.7600 - val_accuracy: 0.7539
Epoch 6/20
90/90 [==============================] - 29s 319ms/step - loss: 1.2422 - accuracy: 0.5506 - val_loss: 0.9704 - val_accuracy: 0.6946
Epoch 7/20
90/90 [==============================] - 29s 318ms/step - loss: 1.2081 - accuracy: 0.5631 - val_loss: 0.8491 - val_accuracy: 0.7

In [ ]:
time.sleep(400)

In [165]:
print(n)#

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
94/94 [==============================] - 31s 310ms/step - loss: 2.0448 - accuracy: 0.3310 - val_loss: 0.9764 - val_accuracy: 0.7152
Epoch 2/20
94/94 [==============================] - 29s 310ms/step - loss: 1.5742 - accuracy: 0.4297 - val_loss: 0.7904 - val_accuracy: 0.7568
Epoch 3/20
94/94 [==============================] - 29s 310ms/step - loss: 1.4878 - accuracy: 0.4534 - val_loss: 0.9564 - val_accuracy: 0.6865
Epoch 4/20
94/94 [==============================] - 29s 312ms/step - loss: 1.4034 - accuracy: 0.4917 - val_loss: 0.8366 - val_accuracy: 0.7435
Epoch 5/20
94/94 [==============================] - 29s 312ms/step - loss: 1.3170 - accuracy: 0.5151 - val_loss: 0.8022 - val_accuracy: 0.7412
Epoch 6/20
94/94 [==============================] - 29s 310ms/step - loss: 1.2279 - accuracy: 0.5682 - val_loss: 0.8729 - val_accuracy: 0.6995
Epoch 7/20
94/94 [==============================] - 29s 311ms/step - loss: 1.1712 - accuracy: 0.5822 - val_loss: 0.7925 - val_accuracy: 0.7

In [166]:
new_model_random_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_random_e2"

i=0

for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_random_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_random_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_random_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_random_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_random_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_random_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_random_e2_6\assets
Model 

In [167]:
try:
    del random_indexes
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

In [168]:
gc.collect()

548934

## Training guided by NC

In [169]:
# NC
nc_values = []
for i in range(1,17):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)

In [170]:
nc_values = nc_values[len(x_train):] 
print(len(nc_values))

6000


In [171]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_adversary_training)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_adversary_training)


In [172]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  300
300
1 :
0  ->  600
600
2 :
0  ->  900
900
3 :
0  ->  1200
1200
4 :
0  ->  1500
1500
5 :
0  ->  1800
1800
6 :
0  ->  2100
2100
7 :
0  ->  2400
2400
8 :
0  ->  2700
2700
9 :
0  ->  3000
3000
10 :
0  ->  3300
3300
11 :
0  ->  3600
3600
12 :
0  ->  3900
3900
13 :
0  ->  4200
4200
14 :
0  ->  4500
4500
15 :
0  ->  4800
4800
16 :
0  ->  5100
5100
17 :
0  ->  5400
5400
18 :
0  ->  5700
5700
19 :
Last
0  ->  6000
6000


In [173]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_nc.append(model)


C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [174]:
n=0
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
5/5 [==============================] - 14s 3s/step - loss: 2.4667 - accuracy: 0.4846 - val_loss: 1.0129 - val_accuracy: 0.7066
Epoch 2/20
5/5 [==============================] - 12s 3s/step - loss: 1.4250 - accuracy: 0.4845 - val_loss: 0.8464 - val_accuracy: 0.7463
Epoch 3/20
5/5 [==============================] - 12s 3s/step - loss: 1.2293 - accuracy: 0.5977 - val_loss: 0.7826 - val_accuracy: 0.7504
Epoch 4/20
5/5 [==============================] - 12s 3s/step - loss: 1.1239 - accuracy: 0.5925 - val_loss: 1.0212 - val_accuracy: 0.6704
Epoch 5/20
5/5 [==============================] - 12s 3s/step - loss: 0.9722 - accuracy: 0.6393 - val_loss: 0.9711 - val_accuracy: 0.7109
Epoch 6/20
5/5 [==============================] - 12s 3s/step - loss: 0.9612 - accuracy: 0.6625 - val_loss: 1.0212 - val_accuracy: 0.7143
Epoch 7/20
5/5 [==============================] - 12s 3s/step - loss: 0.8028 - accuracy: 0.7302 - val_loss: 1.1383 - val_accuracy: 0.7061
Epoch 8/20
5/5 [================

In [175]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
10/10 [==============================] - 14s 1s/step - loss: 2.5994 - accuracy: 0.3703 - val_loss: 0.9908 - val_accuracy: 0.7244
Epoch 2/20
10/10 [==============================] - 12s 1s/step - loss: 1.5506 - accuracy: 0.4199 - val_loss: 0.9270 - val_accuracy: 0.7114
Epoch 3/20
10/10 [==============================] - 12s 1s/step - loss: 1.3802 - accuracy: 0.4950 - val_loss: 0.9149 - val_accuracy: 0.7086
Epoch 4/20
10/10 [==============================] - 12s 1s/step - loss: 1.2731 - accuracy: 0.5311 - val_loss: 0.9154 - val_accuracy: 0.7069
Epoch 5/20
10/10 [==============================] - 12s 1s/step - loss: 1.2035 - accuracy: 0.5901 - val_loss: 1.0684 - val_accuracy: 0.6966
Epoch 6/20
10/10 [==============================] - 12s 1s/step - loss: 1.1985 - accuracy: 0.5669 - val_loss: 0.8513 - val_accuracy: 0.7203
Epoch 7/20
10/10 [==============================] - 12s 1s/step - loss: 0.9598 - accuracy: 0.6625 - val_loss: 1.0700 - val_accuracy: 0.7249
Epoch 8/20
10/10 [

In [176]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
15/15 [==============================] - 15s 930ms/step - loss: 2.8077 - accuracy: 0.3402 - val_loss: 1.0413 - val_accuracy: 0.7080
Epoch 2/20
15/15 [==============================] - 13s 911ms/step - loss: 1.5601 - accuracy: 0.4284 - val_loss: 0.8490 - val_accuracy: 0.7312
Epoch 3/20
15/15 [==============================] - 13s 920ms/step - loss: 1.4871 - accuracy: 0.4723 - val_loss: 1.0798 - val_accuracy: 0.6729
Epoch 4/20
15/15 [==============================] - 13s 902ms/step - loss: 1.3351 - accuracy: 0.5093 - val_loss: 0.9771 - val_accuracy: 0.7006
Epoch 5/20
15/15 [==============================] - 13s 903ms/step - loss: 1.3538 - accuracy: 0.5221 - val_loss: 1.0583 - val_accuracy: 0.6582
Epoch 6/20
15/15 [==============================] - 13s 895ms/step - loss: 1.2550 - accuracy: 0.5716 - val_loss: 1.1298 - val_accuracy: 0.6167
Epoch 7/20
15/15 [==============================] - 13s 904ms/step - loss: 1.1505 - accuracy: 0.5886 - val_loss: 1.0100 - val_accuracy: 0.67

In [177]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
19/19 [==============================] - 17s 822ms/step - loss: 2.7983 - accuracy: 0.3494 - val_loss: 0.8011 - val_accuracy: 0.7588
Epoch 2/20
19/19 [==============================] - 14s 787ms/step - loss: 1.5238 - accuracy: 0.4458 - val_loss: 0.7832 - val_accuracy: 0.7533
Epoch 3/20
19/19 [==============================] - 14s 787ms/step - loss: 1.4158 - accuracy: 0.4777 - val_loss: 0.8906 - val_accuracy: 0.7111
Epoch 4/20
19/19 [==============================] - 15s 797ms/step - loss: 1.2757 - accuracy: 0.5404 - val_loss: 0.8574 - val_accuracy: 0.7267
Epoch 5/20
19/19 [==============================] - 14s 788ms/step - loss: 1.2430 - accuracy: 0.5466 - val_loss: 0.8552 - val_accuracy: 0.7239
Epoch 6/20
19/19 [==============================] - 14s 790ms/step - loss: 1.1178 - accuracy: 0.6021 - val_loss: 0.9452 - val_accuracy: 0.7182
Epoch 7/20
19/19 [==============================] - 15s 801ms/step - loss: 1.0775 - accuracy: 0.6231 - val_loss: 1.0662 - val_accuracy: 0.70

In [178]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
24/24 [==============================] - 18s 688ms/step - loss: 2.5199 - accuracy: 0.3167 - val_loss: 0.8255 - val_accuracy: 0.7442
Epoch 2/20
24/24 [==============================] - 15s 659ms/step - loss: 1.5900 - accuracy: 0.4031 - val_loss: 0.8950 - val_accuracy: 0.7188
Epoch 3/20
24/24 [==============================] - 15s 661ms/step - loss: 1.4801 - accuracy: 0.4659 - val_loss: 0.7935 - val_accuracy: 0.7427
Epoch 4/20
24/24 [==============================] - 15s 657ms/step - loss: 1.3563 - accuracy: 0.5022 - val_loss: 0.8224 - val_accuracy: 0.7301
Epoch 5/20
24/24 [==============================] - 16s 675ms/step - loss: 1.2683 - accuracy: 0.5327 - val_loss: 0.9421 - val_accuracy: 0.7228
Epoch 6/20
24/24 [==============================] - 16s 671ms/step - loss: 1.2235 - accuracy: 0.5661 - val_loss: 0.9688 - val_accuracy: 0.7175
Epoch 7/20
24/24 [==============================] - 15s 659ms/step - loss: 1.1740 - accuracy: 0.5897 - val_loss: 0.9200 - val_accuracy: 0.71

In [179]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
29/29 [==============================] - 19s 602ms/step - loss: 2.4977 - accuracy: 0.3320 - val_loss: 1.1889 - val_accuracy: 0.6457
Epoch 2/20
29/29 [==============================] - 17s 585ms/step - loss: 1.6477 - accuracy: 0.3994 - val_loss: 0.9707 - val_accuracy: 0.6839
Epoch 3/20
29/29 [==============================] - 17s 594ms/step - loss: 1.4770 - accuracy: 0.4746 - val_loss: 1.0498 - val_accuracy: 0.6443
Epoch 4/20
29/29 [==============================] - 17s 598ms/step - loss: 1.4360 - accuracy: 0.4894 - val_loss: 1.2031 - val_accuracy: 0.6975
Epoch 5/20
29/29 [==============================] - 17s 585ms/step - loss: 1.2856 - accuracy: 0.5368 - val_loss: 1.0739 - val_accuracy: 0.6532
Epoch 6/20
29/29 [==============================] - 17s 596ms/step - loss: 1.2284 - accuracy: 0.5701 - val_loss: 1.1962 - val_accuracy: 0.6938
Epoch 7/20
29/29 [==============================] - 17s 586ms/step - loss: 1.2002 - accuracy: 0.5738 - val_loss: 0.9644 - val_accuracy: 0.72

In [180]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
33/33 [==============================] - 20s 556ms/step - loss: 2.4147 - accuracy: 0.3202 - val_loss: 0.8587 - val_accuracy: 0.7452
Epoch 2/20
33/33 [==============================] - 18s 559ms/step - loss: 1.6016 - accuracy: 0.4300 - val_loss: 0.9517 - val_accuracy: 0.6976
Epoch 3/20
33/33 [==============================] - 17s 539ms/step - loss: 1.4864 - accuracy: 0.4606 - val_loss: 0.9071 - val_accuracy: 0.7176
Epoch 4/20
33/33 [==============================] - 17s 537ms/step - loss: 1.4255 - accuracy: 0.4759 - val_loss: 0.9302 - val_accuracy: 0.7130
Epoch 5/20
33/33 [==============================] - 17s 536ms/step - loss: 1.3255 - accuracy: 0.5232 - val_loss: 0.9702 - val_accuracy: 0.6930
Epoch 6/20
33/33 [==============================] - 18s 540ms/step - loss: 1.2463 - accuracy: 0.5608 - val_loss: 0.8665 - val_accuracy: 0.7290
Epoch 7/20
33/33 [==============================] - 17s 537ms/step - loss: 1.2241 - accuracy: 0.5684 - val_loss: 0.8912 - val_accuracy: 0.71

In [181]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
38/38 [==============================] - 20s 502ms/step - loss: 2.4458 - accuracy: 0.3058 - val_loss: 0.8817 - val_accuracy: 0.7082
Epoch 2/20
38/38 [==============================] - 19s 494ms/step - loss: 1.6990 - accuracy: 0.3895 - val_loss: 0.9045 - val_accuracy: 0.6980
Epoch 3/20
38/38 [==============================] - 18s 490ms/step - loss: 1.4920 - accuracy: 0.4531 - val_loss: 1.0933 - val_accuracy: 0.6492
Epoch 4/20
38/38 [==============================] - 18s 489ms/step - loss: 1.4210 - accuracy: 0.4721 - val_loss: 0.9380 - val_accuracy: 0.6900
Epoch 5/20
38/38 [==============================] - 19s 494ms/step - loss: 1.3297 - accuracy: 0.5242 - val_loss: 0.9113 - val_accuracy: 0.7046
Epoch 6/20
38/38 [==============================] - 18s 491ms/step - loss: 1.2722 - accuracy: 0.5573 - val_loss: 0.8864 - val_accuracy: 0.6958
Epoch 7/20
38/38 [==============================] - 18s 492ms/step - loss: 1.1635 - accuracy: 0.5850 - val_loss: 0.9385 - val_accuracy: 0.70

In [182]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
43/43 [==============================] - 22s 469ms/step - loss: 2.4221 - accuracy: 0.3052 - val_loss: 0.9828 - val_accuracy: 0.6976
Epoch 2/20
43/43 [==============================] - 20s 459ms/step - loss: 1.6655 - accuracy: 0.3979 - val_loss: 0.8166 - val_accuracy: 0.7253
Epoch 3/20
43/43 [==============================] - 19s 457ms/step - loss: 1.5633 - accuracy: 0.4324 - val_loss: 0.9272 - val_accuracy: 0.6982
Epoch 4/20
43/43 [==============================] - 19s 458ms/step - loss: 1.4483 - accuracy: 0.4707 - val_loss: 0.8190 - val_accuracy: 0.7377
Epoch 5/20
43/43 [==============================] - 19s 455ms/step - loss: 1.3677 - accuracy: 0.5113 - val_loss: 1.0640 - val_accuracy: 0.6859
Epoch 6/20
43/43 [==============================] - 20s 459ms/step - loss: 1.2764 - accuracy: 0.5515 - val_loss: 0.8182 - val_accuracy: 0.7279
Epoch 7/20
43/43 [==============================] - 20s 463ms/step - loss: 1.2457 - accuracy: 0.5504 - val_loss: 0.9124 - val_accuracy: 0.73

In [183]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
47/47 [==============================] - 23s 455ms/step - loss: 2.3889 - accuracy: 0.2907 - val_loss: 0.9110 - val_accuracy: 0.7162
Epoch 2/20
47/47 [==============================] - 21s 444ms/step - loss: 1.6445 - accuracy: 0.3999 - val_loss: 0.9613 - val_accuracy: 0.6879
Epoch 3/20
47/47 [==============================] - 21s 444ms/step - loss: 1.4726 - accuracy: 0.4705 - val_loss: 0.9414 - val_accuracy: 0.6841
Epoch 4/20
47/47 [==============================] - 21s 448ms/step - loss: 1.4521 - accuracy: 0.4807 - val_loss: 0.8247 - val_accuracy: 0.7284
Epoch 5/20
47/47 [==============================] - 21s 447ms/step - loss: 1.3291 - accuracy: 0.5312 - val_loss: 0.8311 - val_accuracy: 0.7347
Epoch 6/20
47/47 [==============================] - 21s 447ms/step - loss: 1.2269 - accuracy: 0.5586 - val_loss: 0.8493 - val_accuracy: 0.7175
Epoch 7/20
47/47 [==============================] - 21s 455ms/step - loss: 1.1777 - accuracy: 0.5726 - val_loss: 0.8467 - val_accuracy: 0.71

In [184]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
52/52 [==============================] - 24s 437ms/step - loss: 2.3452 - accuracy: 0.2884 - val_loss: 0.8261 - val_accuracy: 0.7263
Epoch 2/20
52/52 [==============================] - 22s 430ms/step - loss: 1.7049 - accuracy: 0.3705 - val_loss: 0.9035 - val_accuracy: 0.7172
Epoch 3/20
52/52 [==============================] - 22s 430ms/step - loss: 1.5221 - accuracy: 0.4468 - val_loss: 0.8874 - val_accuracy: 0.7057
Epoch 4/20
52/52 [==============================] - 22s 428ms/step - loss: 1.4543 - accuracy: 0.4678 - val_loss: 0.9698 - val_accuracy: 0.6856
Epoch 5/20
52/52 [==============================] - 22s 432ms/step - loss: 1.3325 - accuracy: 0.5232 - val_loss: 0.9328 - val_accuracy: 0.6814
Epoch 6/20
52/52 [==============================] - 22s 426ms/step - loss: 1.2923 - accuracy: 0.5423 - val_loss: 0.9413 - val_accuracy: 0.6976
Epoch 7/20
52/52 [==============================] - 23s 437ms/step - loss: 1.1910 - accuracy: 0.5793 - val_loss: 0.9396 - val_accuracy: 0.7

In [185]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
57/57 [==============================] - 25s 422ms/step - loss: 2.1739 - accuracy: 0.3184 - val_loss: 0.9276 - val_accuracy: 0.7283
Epoch 2/20
57/57 [==============================] - 23s 414ms/step - loss: 1.6461 - accuracy: 0.3890 - val_loss: 1.0750 - val_accuracy: 0.6327
Epoch 3/20
57/57 [==============================] - 23s 413ms/step - loss: 1.5189 - accuracy: 0.4292 - val_loss: 0.9681 - val_accuracy: 0.6733
Epoch 4/20
57/57 [==============================] - 23s 413ms/step - loss: 1.4253 - accuracy: 0.4873 - val_loss: 0.8643 - val_accuracy: 0.6990
Epoch 5/20
57/57 [==============================] - 23s 412ms/step - loss: 1.3186 - accuracy: 0.5276 - val_loss: 0.9702 - val_accuracy: 0.6545
Epoch 6/20
57/57 [==============================] - 23s 414ms/step - loss: 1.2110 - accuracy: 0.5579 - val_loss: 0.8908 - val_accuracy: 0.6938
Epoch 7/20
57/57 [==============================] - 23s 414ms/step - loss: 1.1772 - accuracy: 0.5798 - val_loss: 0.8725 - val_accuracy: 0.7

In [186]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
61/61 [==============================] - 27s 413ms/step - loss: 2.4095 - accuracy: 0.2875 - val_loss: 1.0479 - val_accuracy: 0.6530
Epoch 2/20
61/61 [==============================] - 24s 401ms/step - loss: 1.6678 - accuracy: 0.3925 - val_loss: 0.9913 - val_accuracy: 0.6877
Epoch 3/20
61/61 [==============================] - 24s 400ms/step - loss: 1.5394 - accuracy: 0.4523 - val_loss: 1.0197 - val_accuracy: 0.6638
Epoch 4/20
61/61 [==============================] - 24s 402ms/step - loss: 1.4088 - accuracy: 0.4958 - val_loss: 0.8464 - val_accuracy: 0.7071
Epoch 5/20
61/61 [==============================] - 24s 401ms/step - loss: 1.3038 - accuracy: 0.5211 - val_loss: 0.8669 - val_accuracy: 0.7088
Epoch 6/20
61/61 [==============================] - 24s 401ms/step - loss: 1.2739 - accuracy: 0.5551 - val_loss: 0.8115 - val_accuracy: 0.7308
Epoch 7/20
61/61 [==============================] - 24s 399ms/step - loss: 1.2045 - accuracy: 0.5677 - val_loss: 0.8952 - val_accuracy: 0.6

In [187]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
66/66 [==============================] - 27s 393ms/step - loss: 2.2279 - accuracy: 0.2875 - val_loss: 0.8269 - val_accuracy: 0.7540
Epoch 2/20
66/66 [==============================] - 25s 388ms/step - loss: 1.6540 - accuracy: 0.3965 - val_loss: 0.8876 - val_accuracy: 0.7261
Epoch 3/20
66/66 [==============================] - 25s 387ms/step - loss: 1.5376 - accuracy: 0.4471 - val_loss: 0.8272 - val_accuracy: 0.7237
Epoch 4/20
66/66 [==============================] - 26s 392ms/step - loss: 1.4297 - accuracy: 0.4778 - val_loss: 0.8129 - val_accuracy: 0.7215
Epoch 5/20
66/66 [==============================] - 26s 389ms/step - loss: 1.3254 - accuracy: 0.5280 - val_loss: 0.9130 - val_accuracy: 0.7002
Epoch 6/20
66/66 [==============================] - 25s 388ms/step - loss: 1.2957 - accuracy: 0.5279 - val_loss: 0.8888 - val_accuracy: 0.6952
Epoch 7/20
66/66 [==============================] - 25s 387ms/step - loss: 1.1902 - accuracy: 0.5606 - val_loss: 0.7612 - val_accuracy: 0.7

In [188]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
71/71 [==============================] - 30s 393ms/step - loss: 2.2519 - accuracy: 0.3001 - val_loss: 1.0287 - val_accuracy: 0.6852
Epoch 2/20
71/71 [==============================] - 27s 383ms/step - loss: 1.6370 - accuracy: 0.4141 - val_loss: 0.7708 - val_accuracy: 0.7544
Epoch 3/20
71/71 [==============================] - 27s 380ms/step - loss: 1.5155 - accuracy: 0.4502 - val_loss: 0.9839 - val_accuracy: 0.6641
Epoch 4/20
71/71 [==============================] - 27s 380ms/step - loss: 1.4514 - accuracy: 0.4656 - val_loss: 0.8891 - val_accuracy: 0.6920
Epoch 5/20
71/71 [==============================] - 27s 380ms/step - loss: 1.3619 - accuracy: 0.5108 - val_loss: 1.0772 - val_accuracy: 0.6334
Epoch 6/20
71/71 [==============================] - 27s 379ms/step - loss: 1.2695 - accuracy: 0.5566 - val_loss: 0.8650 - val_accuracy: 0.6992
Epoch 7/20
71/71 [==============================] - 27s 382ms/step - loss: 1.1849 - accuracy: 0.5694 - val_loss: 0.9724 - val_accuracy: 0.6

In [189]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
75/75 [==============================] - 30s 378ms/step - loss: 2.2036 - accuracy: 0.3202 - val_loss: 1.0620 - val_accuracy: 0.6772
Epoch 2/20
75/75 [==============================] - 28s 372ms/step - loss: 1.5968 - accuracy: 0.4139 - val_loss: 0.8467 - val_accuracy: 0.7297
Epoch 3/20
75/75 [==============================] - 28s 373ms/step - loss: 1.4463 - accuracy: 0.4638 - val_loss: 0.8686 - val_accuracy: 0.7397
Epoch 4/20
75/75 [==============================] - 28s 374ms/step - loss: 1.3762 - accuracy: 0.4933 - val_loss: 0.9222 - val_accuracy: 0.6892
Epoch 5/20
75/75 [==============================] - 28s 375ms/step - loss: 1.2907 - accuracy: 0.5397 - val_loss: 0.7541 - val_accuracy: 0.7497
Epoch 6/20
75/75 [==============================] - 28s 371ms/step - loss: 1.2598 - accuracy: 0.5453 - val_loss: 0.7481 - val_accuracy: 0.7459
Epoch 7/20
75/75 [==============================] - 28s 372ms/step - loss: 1.2009 - accuracy: 0.5699 - val_loss: 0.7925 - val_accuracy: 0.7

In [190]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
80/80 [==============================] - 31s 370ms/step - loss: 2.0244 - accuracy: 0.3271 - val_loss: 0.8072 - val_accuracy: 0.7629
Epoch 2/20
80/80 [==============================] - 29s 365ms/step - loss: 1.6215 - accuracy: 0.4088 - val_loss: 0.9524 - val_accuracy: 0.7129
Epoch 3/20
80/80 [==============================] - 29s 362ms/step - loss: 1.4897 - accuracy: 0.4474 - val_loss: 0.7632 - val_accuracy: 0.7509
Epoch 4/20
80/80 [==============================] - 29s 362ms/step - loss: 1.4001 - accuracy: 0.4939 - val_loss: 0.7052 - val_accuracy: 0.7616
Epoch 5/20
80/80 [==============================] - 29s 364ms/step - loss: 1.3558 - accuracy: 0.5182 - val_loss: 0.8381 - val_accuracy: 0.7228
Epoch 6/20
80/80 [==============================] - 29s 368ms/step - loss: 1.2659 - accuracy: 0.5422 - val_loss: 0.7879 - val_accuracy: 0.7368
Epoch 7/20
80/80 [==============================] - 29s 362ms/step - loss: 1.2028 - accuracy: 0.5457 - val_loss: 0.7541 - val_accuracy: 0.7

In [191]:
time.sleep(400)

In [192]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
85/85 [==============================] - 22s 248ms/step - loss: 2.0535 - accuracy: 0.3158 - val_loss: 0.9486 - val_accuracy: 0.6923
Epoch 2/20
85/85 [==============================] - 21s 242ms/step - loss: 1.6098 - accuracy: 0.4170 - val_loss: 0.8696 - val_accuracy: 0.7220
Epoch 3/20
85/85 [==============================] - 21s 250ms/step - loss: 1.4889 - accuracy: 0.4480 - val_loss: 0.7810 - val_accuracy: 0.7438
Epoch 4/20
85/85 [==============================] - 24s 278ms/step - loss: 1.4119 - accuracy: 0.4944 - val_loss: 0.8840 - val_accuracy: 0.7096
Epoch 5/20
85/85 [==============================] - 28s 329ms/step - loss: 1.2997 - accuracy: 0.5321 - val_loss: 0.8258 - val_accuracy: 0.7465
Epoch 6/20
85/85 [==============================] - 30s 357ms/step - loss: 1.2472 - accuracy: 0.5449 - val_loss: 0.7889 - val_accuracy: 0.7405
Epoch 7/20
85/85 [==============================] - 30s 357ms/step - loss: 1.1806 - accuracy: 0.5754 - val_loss: 0.8634 - val_accuracy: 0.7

In [ ]:
time.sleep(400)

In [193]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
90/90 [==============================] - 32s 341ms/step - loss: 2.0528 - accuracy: 0.3273 - val_loss: 1.1056 - val_accuracy: 0.6415
Epoch 2/20
90/90 [==============================] - 31s 341ms/step - loss: 1.6737 - accuracy: 0.3822 - val_loss: 0.8862 - val_accuracy: 0.7030
Epoch 3/20
90/90 [==============================] - 30s 340ms/step - loss: 1.4658 - accuracy: 0.4687 - val_loss: 0.8645 - val_accuracy: 0.7153
Epoch 4/20
90/90 [==============================] - 30s 337ms/step - loss: 1.3999 - accuracy: 0.4758 - val_loss: 0.8327 - val_accuracy: 0.7198
Epoch 5/20
90/90 [==============================] - 30s 337ms/step - loss: 1.3214 - accuracy: 0.5242 - val_loss: 0.9317 - val_accuracy: 0.6971
Epoch 6/20
90/90 [==============================] - 30s 338ms/step - loss: 1.2480 - accuracy: 0.5488 - val_loss: 0.8402 - val_accuracy: 0.7172
Epoch 7/20
90/90 [==============================] - 30s 337ms/step - loss: 1.2182 - accuracy: 0.5706 - val_loss: 1.2393 - val_accuracy: 0.5

In [ ]:
time.sleep(400)

In [194]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
94/94 [==============================] - 34s 347ms/step - loss: 2.0765 - accuracy: 0.3263 - val_loss: 0.8025 - val_accuracy: 0.7592
Epoch 2/20
94/94 [==============================] - 32s 341ms/step - loss: 1.5901 - accuracy: 0.4200 - val_loss: 0.7318 - val_accuracy: 0.7706
Epoch 3/20
94/94 [==============================] - 32s 342ms/step - loss: 1.4853 - accuracy: 0.4527 - val_loss: 0.8415 - val_accuracy: 0.7236
Epoch 4/20
94/94 [==============================] - 32s 344ms/step - loss: 1.4119 - accuracy: 0.4845 - val_loss: 0.8155 - val_accuracy: 0.7332
Epoch 5/20
94/94 [==============================] - 32s 344ms/step - loss: 1.3270 - accuracy: 0.5261 - val_loss: 0.8353 - val_accuracy: 0.7270
Epoch 6/20
94/94 [==============================] - 32s 342ms/step - loss: 1.2561 - accuracy: 0.5548 - val_loss: 0.7662 - val_accuracy: 0.7524
Epoch 7/20
94/94 [==============================] - 32s 342ms/step - loss: 1.1923 - accuracy: 0.5597 - val_loss: 0.7704 - val_accuracy: 0.7

In [195]:
new_model_nc_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/e2/"+dataset+"_model_c3_aug_nc_e2"

i=0

for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_nc_e2_0\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_nc_e2_1\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_nc_e2_2\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_nc_e2_3\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_nc_e2_4\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_nc_e2_5\assets
Model has been saved
INFO:tensorflow:Assets written to: C:/Users/fjdurlop/Documents/upc/models/cifar/C3/e2/cifar_model_c3_aug_nc_e2_6\assets
Model has been saved
INFO:tensorfl

In [ ]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [ ]:
gc.collect()

## Evaluating

In [ ]:
model_original.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
# Metrics using adversarial test
evaluate_lsa_0 = []
evaluate_dsa_0 = []
evaluate_nc_0 = []
evaluate_random_0 = []

evaluate_lsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_dsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_nc_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_random_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

for model in models_lsa:
    evaluate_lsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_dsa:
    evaluate_dsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_random:
    evaluate_random_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_nc:
    evaluate_nc_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

In [ ]:
evaluate_lsa_0[0]=[0,0,0,0]
evaluate_dsa_0[0]=[0,0,0,0]
evaluate_random_0[0]=[0,0,0,0]
evaluate_nc_0[0]=[0,0,0,0]

In [ ]:
print(len(evaluate_lsa_0))
print(len(evaluate_dsa_0))
print(len(evaluate_random_0))
print(len(evaluate_nc_0))

In [ ]:
# Metrics using original test
evaluate_lsa_1 = []
evaluate_dsa_1 = []
evaluate_nc_1 = []
evaluate_random_1 = []

evaluate_lsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_dsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_nc_1.append(model_original.evaluate(x_test,y_test))
evaluate_random_1.append(model_original.evaluate(x_test,y_test))
print("lsa---")
for model in models_lsa:
    evaluate_lsa_1.append(model.evaluate(x_test,y_test))

print("dsa---")
    
for model in models_dsa:
    evaluate_dsa_1.append(model.evaluate(x_test,y_test))
    

print("random---")

for model in models_random:
    evaluate_random_1.append(model.evaluate(x_test,y_test))
    
print("nc---")

for model in models_nc:
    evaluate_nc_1.append(model.evaluate(x_test,y_test))


In [ ]:
evaluate_lsa_1[0]=[0,0,0,0]
evaluate_dsa_1[0]=[0,0,0,0]
evaluate_random_1[0]=[0,0,0,0]
evaluate_nc_1[0]=[0,0,0,0]

In [ ]:

print(len(evaluate_lsa_1))
print(len(evaluate_dsa_1))
print(len(evaluate_random_1))
print(len(evaluate_nc_1))

In [ ]:
type(evaluate_dsa_0[0][0])

In [ ]:
import pandas as pd

df_evaluate_lsa_0 = pd.DataFrame(np.array(evaluate_lsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_lsa_1 = pd.DataFrame(np.array(evaluate_lsa_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_0 = pd.DataFrame(np.array(evaluate_dsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_1 = pd.DataFrame(np.array(evaluate_dsa_1),columns=["loss","accuracy","precision","recall"])

df_evaluate_random_0 = pd.DataFrame(np.array(evaluate_random_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_random_1 = pd.DataFrame(np.array(evaluate_random_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_0 = pd.DataFrame(np.array(evaluate_nc_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_1 = pd.DataFrame(np.array(evaluate_nc_1),columns=["loss","accuracy","precision","recall"])


In [ ]:
# Original test set + adversarial set

metric ="accuracy"
accuracy_lsa_3 = (np.array(df_evaluate_lsa_0[metric])+np.array(df_evaluate_lsa_1[metric]))/2
accuracy_dsa_3 = (np.array(df_evaluate_dsa_0[metric])+np.array(df_evaluate_dsa_1[metric]))/2
accuracy_nc_3 = (np.array(df_evaluate_nc_0[metric])+np.array(df_evaluate_nc_1[metric]))/2
accuracy_random_3 = (np.array(df_evaluate_random_0[metric])+np.array(df_evaluate_random_1[metric]))/2


## Charts

In [ ]:
n_inputs = [1800*i for i in range(20)]
n_inputs.append(len(x_train_and_adversary))

In [ ]:
print(n_inputs)

In [ ]:
cd "C:/Users/fjdur/Desktop/gtsrb_graphs/e2_graphs_dec/"

In [ ]:
linestyles = ['solid','dotted','dashed','dashdot']
colors =['k','k','k','k']

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

#metric = "accuracy" # accuracy loss
my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_0[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_0[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_0[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_0[metric],colors[3],linestyle=linestyles[3])


    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with test set of adversarial examples FGSM")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_0.svg")
    plt.show()

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    
#metric = "accuracy" # accuracy loss
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_1[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_1[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_1[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_1[metric],colors[3],linestyle=linestyles[3])

    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with original set")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_1.svg")
    plt.show()

In [ ]:
#adversarial jsma test set
#configuration 3
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.plot(n_inputs,accuracy_lsa_3,colors[0],linestyle=linestyles[0])
plt.plot(n_inputs,accuracy_dsa_3,colors[1],linestyle=linestyles[1])
plt.plot(n_inputs,accuracy_random_3,colors[2],linestyle=linestyles[2])
plt.plot(n_inputs,accuracy_nc_3,colors[3],linestyle=linestyles[3])


legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



plt.legend(handles=legend_elements)#
plt.title("accuracy with both sets")

print(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max())
print(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max())
print(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max())
print(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max())

plt.plot(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max(),'-kD')
plt.plot(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max(),'-ko')
plt.plot(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max(),'-kv')
plt.plot(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max(),'-kp')

plt.xlabel('number of inputs')
plt.ylabel('accuracy')
plt.xlim([0, 37000])
#plt.ylim([0, 1])

plt.savefig("gtsrb_c1_"+"accuracy" + "_both.svg")

plt.show()

## Saving models

In [ ]:
new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_dsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2"

i=0
for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_random_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_random_e2"

i=0
for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_nc_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_nc_e2"

i=0
for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1